In [139]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif, chi2, RFECV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV
import time
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

## Data Exploration

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,Patient_ID,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Deceased
0,1,4696,3,Miss Linda Betty,NaN,0,0,225,NaN,Santa Fe,1
1,2,21436,1,Ms. Ramona Elvira,1966.0,0,1,1663,NaN,Albuquerque,0
2,3,7273,3,Mr. Mario Vernon,1982.0,0,0,221,NaN,Santa Fe,1
3,4,8226,3,Mr. Hector Joe,1997.0,0,0,220,NaN,Santa Fe,1
4,5,19689,3,Ms. Jennie Debra,1994.0,0,0,222,NaN,Santa Fe,0


In [4]:
train.shape

(900, 11)

In [5]:
train.columns

Index(['Patient_ID', 'Family_Case_ID', 'Severity', 'Name', 'Birthday_year',
       'Parents or siblings infected', 'Wife/Husband or children infected',
       'Medical_Expenses_Family', 'Medical_Tent', 'City', 'Deceased'],
      dtype='object')

In [6]:
train.dtypes

Patient_ID                             int64
Family_Case_ID                         int64
Severity                               int64
Name                                  object
Birthday_year                        float64
Parents or siblings infected           int64
Wife/Husband or children infected      int64
Medical_Expenses_Family                int64
Medical_Tent                          object
City                                  object
Deceased                               int64
dtype: object

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Patient_ID                         900 non-null    int64  
 1   Family_Case_ID                     900 non-null    int64  
 2   Severity                           900 non-null    int64  
 3   Name                               900 non-null    object 
 4   Birthday_year                      723 non-null    float64
 5   Parents or siblings infected       900 non-null    int64  
 6   Wife/Husband or children infected  900 non-null    int64  
 7   Medical_Expenses_Family            900 non-null    int64  
 8   Medical_Tent                       198 non-null    object 
 9   City                               898 non-null    object 
 10  Deceased                           900 non-null    int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 77.5+ KB


In [8]:
train.describe()

,Patient_ID,Family_Case_ID,Severity,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Deceased
count,900.000000,900.000000,900.000000,723.000000,900.000000,900.000000,900.000000,900.000000
mean,450.500000,14305.827778,2.313333,1990.266943,0.380000,0.521111,895.743333,0.614444
std,259.951919,25418.153900,0.834223,14.523335,0.803247,1.098385,1385.829926,0.486997
min,1.000000,345.000000,1.000000,1940.000000,0.000000,0.000000,0.000000,0.000000
25%,225.750000,8203.000000,2.000000,1982.000000,0.000000,0.000000,221.000000,0.000000
50%,450.500000,13593.500000,3.000000,1992.000000,0.000000,0.000000,405.000000,1.000000
75%,675.250000,18906.500000,3.000000,1999.500000,0.000000,1.000000,861.250000,1.000000
max,900.000000,742836.000000,3.000000,2019.000000,6.000000,8.000000,14345.000000,1.000000


In [9]:
for col in ['Severity','Birthday_year','Medical_Tent','City']:
    print(col + ':', train[col].unique())

Severity: [3 1 2]
Birthday_year: [  nan 1966. 1982. 1997. 1994. 1984. 1989. 2000. 2009. 1999. 1991. 2019.
 1964. 2017. 1987. 2011. 1998. 2002. 1992. 1981. 1990. 1978. 2001. 1958.
 1996. 2004. 1962. 1970. 1985. 1965. 2018. 1995. 1956. 2003. 1986. 1971.
 1983. 2005. 1975. 1950. 2016. 2013. 1980. 1979. 2014. 1988. 1993. 1968.
 2006. 1973. 2007. 2012. 1946. 1974. 1972. 2008. 1976. 1961. 1957. 1960.
 1959. 2015. 2010. 1969. 1949. 1955. 1977. 1963. 1954. 1967. 1940.]
Medical_Tent: [nan 'F' 'B' 'A' 'E' 'C' 'D' 'G' 'T']
City: ['Santa Fe' 'Albuquerque' 'Taos' nan]


In [10]:
train.duplicated().any()

False

In [11]:
train.groupby(['Deceased']).count()

,Patient_ID,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City
Deceased,,,,,,,,,,
0,347,347,347,347,295,347,347,347,131,345
1,553,553,553,553,428,553,553,553,67,553


In [12]:
train.columns

Index(['Patient_ID', 'Family_Case_ID', 'Severity', 'Name', 'Birthday_year',
       'Parents or siblings infected', 'Wife/Husband or children infected',
       'Medical_Expenses_Family', 'Medical_Tent', 'City', 'Deceased'],
      dtype='object')

In [13]:
data = train.drop(['Patient_ID','Deceased'],1)
target = train['Deceased']

Only three columns: Severity, Medical_Tent, and City, are categorical variables, and only Name, Medical_Tent, and City are non-numerical.

The values for all variables seem to be within the expected range.

There are null values in the columns: Birthday_year, Medical_Tent, and City.

There are no duplicated lines.

The target classes are not balanced.

## Preprocessing

### Add New Variables

`Infected_per_Family` Sum of Parents or sibilings infected and Wife/Husband or children infected<br>
`Maritial_state` 1 if it is married 0 otherwise <br>
`Gender` 1 if it is female 0 otherwise <br>
`Is_young: `0 if the age is lower than 60, 1 otherwise<br>

In [14]:
data['Infected_per_Family'] = data['Parents or siblings infected'] + data['Wife/Husband or children infected']
data['Marital_Status'] = 0
data['Gender'] = 0
data['Title'] = 0
# 2020-60 = 1960
data['Is_young'] = 0
for i in data.index:
    if data['Birthday_year'][i]<=1960:
        data['Is_young'][i] = 0
    elif data['Birthday_year'][i]>1960:
        data['Is_young'][i] = 1
    else:
        data['Is_young'][i] = None

In [15]:
for i in data.index:
    if data['Name'][i].startswith('Miss'):
        data['Marital_Status'][i] = 0
        data['Gender'][i] = 1
        data['Title'][i] = 'Miss'
        
    elif data['Name'][i].startswith('Ms.'):
        data['Marital_Status'][i] = None
        data['Gender'][i] = 1
        data['Title'][i] = 'Ms.'
        
    elif data['Name'][i].startswith('Mrs.'):
        data['Marital_Status'][i] = 1
        data['Gender'][i] = 1
        data['Title'][i] = 'Mrs.'
        
    elif data['Name'][i].startswith('Mr.'):
        data['Marital_Status'][i] = 1
        data['Gender'][i] = 0
        data['Title'][i] = 'Mr.'
        
    elif data['Name'][i].startswith('Master'):
        data['Marital_Status'][i] = 0
        data['Gender'][i] = 0
        data['Title'][i] = 'Master'

In [16]:
data[data['Marital_Status'].isnull()].shape

(209, 14)

### Encoding Categorical Values

Because the label encoder does not work with null values in the data frame, but we need the variables to be encoded to use a KNNImputer, first we are replacing the all null value in the City column with a dummy, absurd variable, run the encoder, and then turn them back to nulls.

In [17]:
data[['City']] = data[['City']].fillna('nan')

In [18]:
nan = data[data['City'] == 'nan'].index

In [19]:
le_city = LabelEncoder()
le_title = LabelEncoder()

In [20]:
le_city.fit(data['City'])
le_title.fit(data['Title'])

LabelEncoder()

In [21]:
data['City'] = le_city.transform(data['City'])
data['Title'] = le_title.transform(data['Title'])

In [22]:
data.loc[nan,['City']] = np.nan

In [23]:
data.iloc[nan]

,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Infected_per_Family,Marital_Status,Gender,Title,Is_young
49,23155,1,Ms. Elizabeth Teri,1958.0,0,0,2240,B,NaN,0,NaN,1,3,0.0
530,23155,1,Ms. Tricia Kathryn,1982.0,0,0,2240,B,NaN,0,NaN,1,3,1.0


### Null Values

There were null values in the columns Birthday_year, City and Medical_Tent.

- **Birthday_year:** 117 (13% of the data)
- **Medical_Tent:** 702 (78% of the data)
- **City:** 2

And the new column Marital_Status also has null values:

- **Marital_Status:** 209 (23% of the data)

In [24]:
data[data.isnull().any(axis=1)]

,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Infected_per_Family,Marital_Status,Gender,Title,Is_young
0,4696,3,Miss Linda Betty,NaN,0,0,225,NaN,1.0,0,0.0,1,1,NaN
1,21436,1,Ms. Ramona Elvira,1966.0,0,1,1663,NaN,0.0,1,NaN,1,3,1.0
2,7273,3,Mr. Mario Vernon,1982.0,0,0,221,NaN,1.0,0,1.0,0,2,1.0
3,8226,3,Mr. Hector Joe,1997.0,0,0,220,NaN,1.0,0,1.0,0,2,1.0
4,19689,3,Ms. Jennie Debra,1994.0,0,0,222,NaN,1.0,0,NaN,1,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,6253,3,Ms. Linda Wilcox,1998.0,1,1,344,NaN,1.0,2,NaN,1,3,1.0
896,6483,3,Mr. Haiden Vance,2006.0,0,0,258,NaN,1.0,0,1.0,0,2,1.0
897,981,3,Miss Anaiya Love,1990.0,0,0,214,NaN,2.0,0,0.0,1,1,1.0
898,16418,2,Mr. Robert Williams,1994.0,1,1,812,NaN,1.0,2,1.0,0,2,1.0


In [25]:
data[data.isnull().any(axis=1)].shape

(796, 14)

In [26]:
null_ind = data[data.isnull().any(axis=1)].index

In [27]:
len(null_ind)

796

There are 78% of the data missing on the Medical_Tent column, this is too much therefore this column will be dropped.

Since there are only two null values in the column City, they can simply be replaced using the mode.

When it came to replacing null the values in the Marital_Status column the following rule was defined:

- if the family has a Mr. in it and no Mrs. then we consider the person married, otherwise, we consider the person to be single

Using this data, the null values in the Birthday_year column were filled using the following logic:

- random numbers based on Maritial_Status and Title
- the marrige average for 2019 in the USA to women was 27y and to men 29y
- `Mrs. and Ms. married` random number between 2002 and 1940<br>
- `Mr.` random number between 2002 and 1940<br>
- `Master` random number between 2003 and 2019<br>
- `Ms. not married` random number between 2002 and 1940<br>
- `Miss if child` random number between 2003 and 2019<br>
- `Miss if not child` random number between 2002 and 1940<br>

In [28]:
def null_values(data, method='knn', k=1): 
    # drop the Medical_Tent column 
    data = data.drop(['Medical_Tent'],1)
    
    if method == 'knn':
        imputer = KNNImputer(n_neighbors=k)
        imputer.fit(data)
        data = pd.DataFrame(data=imputer.transform(data),columns=data.columns,index=data.index)
        
    if method == 'stat':
        cols = []
        for col in ['Birthday_year','City','Marital_Status','Is_young']:
            if col in data.columns:
                cols.append(col)
                
        for col in cols:
            if col == 'Birthday_year':
                data[col] = data[col].fillna(round(data[col].mean()))
            else:
                data[col] = data[col].fillna(float(data[col].mode()))
    
    return data

In [29]:
null_values(data.drop(['Name'],1)).iloc[null_ind,:]

,Family_Case_ID,Severity,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,City,Infected_per_Family,Marital_Status,Gender,Title,Is_young
0,4696.0,3.0,1991.0,0.0,0.0,225.0,1.0,0.0,0.0,1.0,1.0,1.0
1,21436.0,1.0,1966.0,0.0,1.0,1663.0,0.0,1.0,1.0,1.0,3.0,1.0
2,7273.0,3.0,1982.0,0.0,0.0,221.0,1.0,0.0,1.0,0.0,2.0,1.0
3,8226.0,3.0,1997.0,0.0,0.0,220.0,1.0,0.0,1.0,0.0,2.0,1.0
4,19689.0,3.0,1994.0,0.0,0.0,222.0,1.0,0.0,1.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
895,6253.0,3.0,1998.0,1.0,1.0,344.0,1.0,2.0,0.0,1.0,3.0,1.0
896,6483.0,3.0,2006.0,0.0,0.0,258.0,1.0,0.0,1.0,0.0,2.0,1.0
897,981.0,3.0,1990.0,0.0,0.0,214.0,2.0,0.0,0.0,1.0,1.0,1.0
898,16418.0,2.0,1994.0,1.0,1.0,812.0,1.0,2.0,1.0,0.0,2.0,1.0


### Outliers

Since all categorical variables have values within the expected range, we're going to check only the continuous variables for outliers.

In [30]:
data.columns

Index(['Family_Case_ID', 'Severity', 'Name', 'Birthday_year',
       'Parents or siblings infected', 'Wife/Husband or children infected',
       'Medical_Expenses_Family', 'Medical_Tent', 'City',
       'Infected_per_Family', 'Marital_Status', 'Gender', 'Title', 'Is_young'],
      dtype='object')

In [31]:
col = ['Birthday_year','Parents or siblings infected','Wife/Husband or children infected','Medical_Expenses_Family','Infected_per_Family']

for c in col:
    sns.boxplot(data=data[c],orient='h',whis=3.5)
    plt.ylabel(c)
    plt.show()

In [32]:
def outliers(data, target, std=3, remove=False, pr=False):
    cols = []
    for col in ['Birthday_year','Parents or siblings infected','Wife/Husband or children infected','Medical_Expenses_Family','Infected_per_Family']:
        if col in data.columns:
            cols.append(col)
    
    ind = []
    ind_to_remove = []
    
    if len(cols)>0:
        out = data[cols]
        out = pd.DataFrame(data=StandardScaler().fit_transform(out),index=out.index,columns=out.columns)

        for col in out.columns:
            ind_col = list(out[(out[col]<-std)|(out[col]>std)].index)
            ind += ind_col

        for i in ind:
            if i not in ind_to_remove:
                ind_to_remove.append(i)
            
    if pr == True:
        print(len(ind_to_remove))
    
    if remove == True:
        for i in ind_to_remove:
            data = data.drop(i,0)
            target = target.drop(i,0)
        
        return data, target

In [33]:
outliers(data, target, pr=True)

66


This is around 7% of the data, and so an acceptable number of outliers to remove.

These outliers can be ignored or removed.

### Oversampling

Since the target variable is unbalanced, a hypothesis is to balance the dataset through oversampling. Undersampling was not considered because there's not much data to begin with.

In [34]:
def oversample(data, target):
    X = pd.concat([data, target], axis=1)      

    false = X[X.Deceased==0]
    true = X[X.Deceased==1]    

    false_upsampled = resample(false,
                               replace=True, # sample with replacement
                               n_samples=len(medium), # match number in majority class
                               random_state=27) # reproducible results        

    upsampled = pd.concat([false_upsampled, true])

    data = upsampled.drop(['Deceased'],1)
    target = upsampled['Deceased']   

    return data, target

## Tuning Hyperparameters

In [164]:
def avg_score(model, method='knn', k=1, remove=False, std=3, oversampling=False, scaler=MinMaxScaler(), grid_search=False, parameters=None, features=False):
    # apply kfold
    kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    
    # create lists to store the results from the different models 
    score_train = []
    score_test = []
    timer = []
    n_iter = []
    
    data__ = data.drop('Name',1)
    
    """cat_col = []
    num_col = []

    for col in data__.columns:
        if col in ['Severity','City','Marital_Status','Gender','Title','Is_young']:
            cat_col.append(col)
        elif col in ['Family_Case_ID','Birthday_year','Parents or siblings infected','Wife/Husband or children infected','Medical_Expenses_Family','Infected_per_Family']:
            num_col.append(col)
    
    anova = pd.Series(data=0,index=num_col)
    chi = pd.Series(data=0,index=cat_col)"""
    
    for train_index, test_index in kf.split(data__,target):
        # get the indexes of the observations assigned for each partition
        X_train, X_test = data__.iloc[train_index], data__.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]
        
        # deal with the null values
        X_train = null_values(X_train, method=method, k=k)
        X_test = null_values(X_test, method=method,k=k)
        
        # deal with outliers
        if remove == True:
            X_train, y_train = outliers(X_train, y_train, std=std, remove=remove)
        
        # oversample the minority class
        if oversampling == True:
            X_train, y_train = oversample(X_train, y_train)
            
        """if features == True:
            data_cat = X_train[cat_col]
            data_num = X_train[num_col]
            
            anova_ = pd.Series(data=f_classif(data_num,y_train)[0],index=data_num.columns)
            chi_ = pd.Series(data=chi2(data_cat,y_train)[0],index=data_cat.columns)
            
            anova += anova_
            chi += chi_"""
            
        # scale the data
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # start counting time
        begin = time.perf_counter()
        
        # find the best features
        if features == True:
            rfe = RFECV(estimator = model,n_jobs=-1,scoring='accuracy')
            X_train = rfe.fit_transform(X_train,y_train)
            X_test = rfe.transform(X_test)
        
        # find the best hyperparameters
        if grid_search == True:
            clf = GridSearchCV(model,parameters,n_jobs=-1,scoring='accuracy',random_state=0)
            clf.fit(X_train,y_train)
            model = clf.best_estimator_
            print(clf.best_params_)
            
        # fit the model to the data
        model.fit(X_train, y_train)
        
        # finish counting time
        end = time.perf_counter()
        
        # predictions
        Y_pred = model.predict(X_train)
        y_pred = model.predict(X_test)
        
        # check the mean f1-score for the train
        value_train = model.score(X_train,y_train)
        
        # check the mean f1-score for the test
        value_test = model.score(X_test,y_test)
        
        # print a classification report and confusion matrix
        print(classification_report(y_true=y_test,y_pred=y_pred))
        print(confusion_matrix(y_true=y_test,y_pred=y_pred))
        
        # append the scores, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_test.append(value_test)
        timer.append(end-begin)
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_test = round(np.mean(score_test),3)
    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_test = round(np.std(score_test),2)
    
    """if features == True:
        anova = anova.sort_values(ascending=False)
        chi = chi.sort_values(ascending=False)
        
        print(anova)
        print(chi)"""
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test)

In [36]:
def show_results(df, models, method = 'knn', k = 1, remove = False, std = 3, oversampling = False, scaler = MinMaxScaler(), parameters = None,features=False):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    for model in models:
        parameter_space = None
        if parameters != None:
            parameter_space = parameters[models.index(model)]
            
        if parameter_space == None:
            grid_search = False
        else:
            grid_search = True
                
        print('----------------------------------')
        # obtain the results provided by avg_score
        time, avg_train, avg_test = avg_score(model, method = method, k = k, remove = remove, std = std, oversampling = oversampling, grid_search = grid_search, scaler = scaler, parameters = parameter_space, features = features)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test
        count+=1

    return df

### KNeighborsClassifier(n_neighbors)

In [37]:
model_1 = KNeighborsClassifier(n_neighbors=1)
model_2 = KNeighborsClassifier(n_neighbors=2)
model_3 = KNeighborsClassifier(n_neighbors=3)
model_4 = KNeighborsClassifier(n_neighbors=4)
model_5 = KNeighborsClassifier(n_neighbors=5)
model_6 = KNeighborsClassifier(n_neighbors=6)
model_7 = KNeighborsClassifier(n_neighbors=7)
model_8 = KNeighborsClassifier(n_neighbors=8)
model_9 = KNeighborsClassifier(n_neighbors=9)
model_10 = KNeighborsClassifier(n_neighbors=10)

In [38]:
df = pd.DataFrame(columns=['Time','Train','Test'],index=range(1,11))
show_results(df,[model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10])

----------------------------------
              precision    recall  f1-score   support

           0       0.67      0.64      0.65        69
           1       0.78      0.80      0.79       111

    accuracy                           0.74       180
   macro avg       0.72      0.72      0.72       180
weighted avg       0.74      0.74      0.74       180

[[44 25]
 [22 89]]
              precision    recall  f1-score   support

           0       0.63      0.64      0.63        69
           1       0.77      0.77      0.77       111

    accuracy                           0.72       180
   macro avg       0.70      0.70      0.70       180
weighted avg       0.72      0.72      0.72       180

[[44 25]
 [26 85]]
              precision    recall  f1-score   support

           0       0.68      0.68      0.68        69
           1       0.80      0.80      0.80       111

    accuracy                           0.76       180
   macro avg       0.74      0.74      0.74       180
w

              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
----------------------------------
              precision    recall  f1-score   support

           0       0.72      0.71      0.72        69
           1       0.82      0.83      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[49 20]
 [19 92]]
              precision    recall  f1-score   support

           0       0.70      0.71      0.71        69
           1       0.82      0.81      0.81       111

    accuracy                           0.77       180
   macro avg       0.76      0.76      0.76       1

              precision    recall  f1-score   support

           0       0.75      0.74      0.75        70
           1       0.84      0.85      0.84       110

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[52 18]
 [17 93]]
              precision    recall  f1-score   support

           0       0.77      0.59      0.67        70
           1       0.77      0.89      0.83       110

    accuracy                           0.77       180
   macro avg       0.77      0.74      0.75       180
weighted avg       0.77      0.77      0.76       180

[[41 29]
 [12 98]]


,Time,Train,Test
1,0.004+/-0.0,0.999+/-0.0,0.747+/-0.02
2,0.003+/-0.0,0.872+/-0.01,0.707+/-0.02
3,0.003+/-0.0,0.877+/-0.0,0.797+/-0.04
4,0.003+/-0.0,0.859+/-0.01,0.792+/-0.04
5,0.003+/-0.0,0.853+/-0.01,0.809+/-0.03
6,0.002+/-0.0,0.837+/-0.01,0.799+/-0.02
7,0.002+/-0.0,0.841+/-0.01,0.807+/-0.02
8,0.003+/-0.0,0.833+/-0.01,0.797+/-0.01
9,0.002+/-0.0,0.834+/-0.01,0.799+/-0.02
10,0.002+/-0.0,0.828+/-0.01,0.797+/-0.02


In [39]:
model_10 = KNeighborsClassifier(n_neighbors=10)
model_20 = KNeighborsClassifier(n_neighbors=20)
model_30 = KNeighborsClassifier(n_neighbors=30)
model_40 = KNeighborsClassifier(n_neighbors=40)
model_50 = KNeighborsClassifier(n_neighbors=50)
model_60 = KNeighborsClassifier(n_neighbors=60)
model_70 = KNeighborsClassifier(n_neighbors=70)
model_80 = KNeighborsClassifier(n_neighbors=80)
model_90 = KNeighborsClassifier(n_neighbors=90)
model_100 = KNeighborsClassifier(n_neighbors=100)

In [40]:
df = pd.DataFrame(columns=['Time','Train','Test'],index=range(10,110,10))
show_results(df,[model_10,model_20,model_30,model_40,model_50,model_60,model_70,model_80,model_90,model_100])

----------------------------------
              precision    recall  f1-score   support

           0       0.75      0.72      0.74        69
           1       0.83      0.85      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.80      0.80      0.80       180

[[50 19]
 [17 94]]
              precision    recall  f1-score   support

           0       0.70      0.72      0.71        69
           1       0.83      0.81      0.82       111

    accuracy                           0.78       180
   macro avg       0.76      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[50 19]
 [21 90]]
              precision    recall  f1-score   support

           0       0.80      0.74      0.77        69
           1       0.84      0.88      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.82       180
w


[[51 19]
 [15 95]]
              precision    recall  f1-score   support

           0       0.83      0.63      0.72        70
           1       0.80      0.92      0.85       110

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 44  26]
 [  9 101]]
----------------------------------
              precision    recall  f1-score   support

           0       0.78      0.65      0.71        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [13 98]]
              precision    recall  f1-score   support

           0       0.71      0.68      0.70        69
           1       0.81      0.83      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.

              precision    recall  f1-score   support

           0       0.76      0.73      0.74        70
           1       0.83      0.85      0.84       110

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[51 19]
 [16 94]]
              precision    recall  f1-score   support

           0       0.79      0.64      0.71        70
           1       0.80      0.89      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 25]
 [12 98]]


,Time,Train,Test
10,0.003+/-0.0,0.828+/-0.01,0.797+/-0.02
20,0.003+/-0.0,0.822+/-0.01,0.791+/-0.02
30,0.003+/-0.0,0.817+/-0.01,0.792+/-0.01
40,0.003+/-0.0,0.802+/-0.0,0.783+/-0.01
50,0.004+/-0.0,0.795+/-0.01,0.79+/-0.02
60,0.003+/-0.0,0.792+/-0.0,0.79+/-0.02
70,0.003+/-0.0,0.791+/-0.0,0.79+/-0.02
80,0.003+/-0.0,0.789+/-0.0,0.79+/-0.02
90,0.003+/-0.0,0.789+/-0.0,0.79+/-0.02
100,0.003+/-0.0,0.788+/-0.0,0.787+/-0.01


In [41]:
model_uniform = KNeighborsClassifier(weights='uniform')
model_distance = KNeighborsClassifier(weights='distance')

df = pd.DataFrame(columns=['Time','Train','Test'],index=['uniform','distance'])
show_results(df,[model_uniform,model_distance])

----------------------------------
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
              precision    recall  f1-score   support

           0       0.71      0.65      0.68        69
           1       0.79      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.77      0.76       180

[[45 24]
 [18 93]]
              precision    recall  f1-score   support

           0       0.82      0.71      0.76        69
           1       0.83      0.90      0.87       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
w

,Time,Train,Test
uniform,0.003+/-0.0,0.853+/-0.01,0.809+/-0.03
distance,0.003+/-0.0,0.999+/-0.0,0.793+/-0.02


In [42]:
model_ball = KNeighborsClassifier(algorithm='ball_tree')
model_kd = KNeighborsClassifier(algorithm='kd_tree')
model_brute = KNeighborsClassifier(algorithm='brute')

df = pd.DataFrame(columns=['Time','Train','Test'],index=['ball tree','kd tree','brute-force'])
show_results(df,[model_ball,model_kd,model_brute])

----------------------------------
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
              precision    recall  f1-score   support

           0       0.71      0.65      0.68        69
           1       0.79      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.77      0.76       180

[[45 24]
 [18 93]]
              precision    recall  f1-score   support

           0       0.82      0.71      0.76        69
           1       0.83      0.90      0.87       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
w

,Time,Train,Test
ball tree,0.002+/-0.0,0.853+/-0.01,0.809+/-0.03
kd tree,0.003+/-0.0,0.853+/-0.01,0.809+/-0.03
brute-force,0.001+/-0.0,0.853+/-0.01,0.809+/-0.03


In [43]:
model_1 = KNeighborsClassifier(p=1)
model_2 = KNeighborsClassifier(p=2)
model_3 = KNeighborsClassifier(p=3)
model_cosine = KNeighborsClassifier(metric='cosine')

df = pd.DataFrame(columns=['Time','Train','Test'],index=list(range(1,4))+['cosine'])
show_results(df,[model_1,model_2,model_3,model_cosine])

----------------------------------
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
              precision    recall  f1-score   support

           0       0.84      0.74      0.78        69
           1       0.85      0.91      0.88       111

    accuracy                           0.84       180
   macro avg       0.84      0.82      0.83       180
w

,Time,Train,Test
1,0.004+/-0.0,0.859+/-0.01,0.818+/-0.02
2,0.003+/-0.0,0.853+/-0.01,0.809+/-0.03
3,0.002+/-0.0,0.852+/-0.01,0.803+/-0.03
cosine,0.001+/-0.0,0.851+/-0.0,0.814+/-0.03


In [44]:
parameter_space = {
    'n_neighbors':[10,20,30,40,50,60,70,80,90,100],
    'weights':['distance'],
    'algorithm':['ball_tree','kd_tree','brute'],
    'metric':['minkowski','cosine'],
    'p':[1]
}

In [45]:
parameter_space = {
    'n_neighbors':[10,20,30,40,50,60,70,80,90,100],
    'weights':['distance'],
    'algorithm':['ball_tree','kd_tree','brute'],
    'metric':['minkowski','cosine'],
    'p':[1]
}

model = KNeighborsClassifier()

## Tests

### Data Cleaning and Scaling

Given that there are so many alternatives on how to clean and scale the data, they're going to be tested against each other.

In [46]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],scaler=MinMaxScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.64      0.69        69
           1       0.80      0.87      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[44 25]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.606+/-0.97,0.999+/-0.0,0.802+/-0.02


In [62]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.65      0.71        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.139+/-0.01,0.999+/-0.0,0.811+/-0.01


In [63]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.59      0.68        69
           1       0.78      0.90      0.84       111

    accuracy                           0.78       180
   macro avg       0.78      0.75      0.76       180
weighted avg       0.78      0.78      0.78       180

[[ 41  28]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.167+/-0.04,0.999+/-0.0,0.802+/-0.01


In [64]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.68      0.75        69
           1       0.82      0.91      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[ 47  22]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.65      0.70        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[45 24]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski',

,Time,Train,Test
model,0.144+/-0.01,0.999+/-0.0,0.812+/-0.03


In [47]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,std=3,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.12+/-0.0,0.999+/-0.0,0.803+/-0.02


In [66]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.65      0.70        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[45 24]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.133+/-0.01,0.999+/-0.0,0.798+/-0.01


In [67]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.62      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.75       180
weighted avg       0.77      0.77      0.77       180

[[43 26]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.14+/-0.01,0.999+/-0.0,0.797+/-0.02


In [68]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        69
           1       0.81      0.85      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.136+/-0.01,0.999+/-0.0,0.807+/-0.02


In [69]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,std=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.81      0.70      0.75        69
           1       0.83      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[ 48  21]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.133+/-0.01,0.999+/-0.0,0.806+/-0.02


In [70]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.64      0.70        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[44 25]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.14+/-0.01,0.999+/-0.0,0.809+/-0.01


In [71]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.61      0.69        69
           1       0.79      0.90      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 42  27]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.138+/-0.01,0.999+/-0.0,0.804+/-0.01


In [72]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.68      0.74        69
           1       0.82      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[ 47  22]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.148+/-0.01,0.999+/-0.0,0.807+/-0.02


In [73]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.64      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[44 25]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.135+/-0.01,0.999+/-0.0,0.798+/-0.02


In [74]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.15+/-0.03,0.999+/-0.0,0.814+/-0.01


In [75]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.61      0.69        69
           1       0.79      0.90      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 42  27]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.1+/-0.01,0.999+/-0.0,0.806+/-0.01


In [76]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.70      0.75        69
           1       0.83      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[ 48  21]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.104+/-0.0,0.999+/-0.0,0.812+/-0.02


In [77]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        69
           1       0.83      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.79       180
weighted avg       0.80      0.80      0.80       180

[[49 20]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.103+/-0.0,0.999+/-0.0,0.799+/-0.02


In [78]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.106+/-0.01,0.999+/-0.0,0.801+/-0.01


In [79]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        69
           1       0.81      0.85      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.64      0.69        69
           1       0.80      0.87      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[44 25]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.108+/-0.0,0.999+/-0.0,0.799+/-0.01


In [48]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,std=3,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.72      0.75        69
           1       0.84      0.87      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[50 19]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.125+/-0.0,0.999+/-0.0,0.816+/-0.02


In [81]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,std=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.109+/-0.01,0.999+/-0.0,0.803+/-0.02


In [82]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.68      0.73        69
           1       0.82      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.11+/-0.0,0.999+/-0.0,0.811+/-0.01


In [83]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.62      0.70        69
           1       0.79      0.90      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[ 43  26]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.111+/-0.01,0.999+/-0.0,0.807+/-0.01


In [84]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.11+/-0.01,0.999+/-0.0,0.813+/-0.02


In [85]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.106+/-0.0,0.999+/-0.0,0.799+/-0.01


In [86]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.111+/-0.0,0.999+/-0.0,0.81+/-0.02


In [87]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.59      0.67        69
           1       0.78      0.89      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.74      0.75       180
weighted avg       0.78      0.78      0.77       180

[[41 28]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.113+/-0.01,0.999+/-0.0,0.797+/-0.01


In [88]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[48 21]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.108+/-0.01,0.999+/-0.0,0.811+/-0.02


In [89]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.78      0.72      0.75        69
           1       0.84      0.87      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[50 19]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.105+/-0.0,0.999+/-0.0,0.806+/-0.02


In [90]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.77      0.76        69
           1       0.85      0.84      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[53 16]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.107+/-0.01,0.999+/-0.0,0.801+/-0.01


In [91]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        69
           1       0.80      0.85      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.62      0.67        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.75       180
weighted avg       0.76      0.77      0.76       180

[[43 26]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.103+/-0.0,0.999+/-0.0,0.79+/-0.02


In [92]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.72      0.75        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[50 19]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.104+/-0.0,0.999+/-0.0,0.809+/-0.02


In [93]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,std=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.109+/-0.01,0.999+/-0.0,0.807+/-0.01


In [94]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        69
           1       0.84      0.89      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       180

[[50 19]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.65      0.71        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.113+/-0.0,0.999+/-0.0,0.811+/-0.01


In [95]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.61      0.68        69
           1       0.78      0.88      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.75       180
weighted avg       0.78      0.78      0.77       180

[[42 27]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.105+/-0.01,0.999+/-0.0,0.8+/-0.02


In [96]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=3,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.108+/-0.01,0.999+/-0.0,0.812+/-0.02


In [97]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        69
           1       0.80      0.85      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.106+/-0.01,0.999+/-0.0,0.794+/-0.01


In [98]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.107+/-0.01,0.999+/-0.0,0.807+/-0.01


In [99]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.59      0.67        69
           1       0.78      0.89      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.74      0.75       180
weighted avg       0.78      0.78      0.77       180

[[41 28]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.12+/-0.01,0.999+/-0.0,0.806+/-0.02


In [100]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.11+/-0.0,0.999+/-0.0,0.801+/-0.02


In [101]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.83      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[49 20]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        69
           1       0.83      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.79       180
weighted avg       0.80      0.80      0.80       180

[[49 20]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.107+/-0.0,0.999+/-0.0,0.802+/-0.01


In [102]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.75      0.75        69
           1       0.85      0.84      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.80      0.79       180
weighted avg       0.81      0.81      0.81       180

[[52 17]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        69
           1       0.83      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.79       180
weighted avg       0.80      0.80      0.80       180

[[49 20]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.11+/-0.0,0.999+/-0.0,0.8+/-0.01


In [103]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.64      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[44 25]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.107+/-0.0,0.999+/-0.0,0.796+/-0.02


In [104]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.72      0.75        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[50 19]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.107+/-0.01,0.999+/-0.0,0.802+/-0.02


In [105]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,std=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.107+/-0.0,0.999+/-0.0,0.806+/-0.01


In [106]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.62      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.75       180
weighted avg       0.77      0.77      0.77       180

[[43 26]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.109+/-0.0,0.999+/-0.0,0.806+/-0.02


In [107]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.61      0.68        69
           1       0.78      0.88      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.75       180
weighted avg       0.78      0.78      0.77       180

[[42 27]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.12+/-0.0,0.999+/-0.0,0.801+/-0.02


In [108]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=4,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.107+/-0.0,0.999+/-0.0,0.806+/-0.02


In [109]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        69
           1       0.80      0.85      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.106+/-0.01,0.999+/-0.0,0.796+/-0.02


In [110]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.64      0.69        69
           1       0.80      0.87      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[44 25]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.112+/-0.01,0.999+/-0.0,0.809+/-0.02


In [111]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.59      0.67        69
           1       0.78      0.89      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.74      0.75       180
weighted avg       0.78      0.78      0.77       180

[[41 28]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.112+/-0.01,0.999+/-0.0,0.803+/-0.02


In [112]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.116+/-0.01,0.999+/-0.0,0.801+/-0.01


In [113]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        69
           1       0.82      0.85      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algori

,Time,Train,Test
model,0.104+/-0.0,0.999+/-0.0,0.803+/-0.02


In [114]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.70      0.72        69
           1       0.82      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.107+/-0.0,0.999+/-0.0,0.797+/-0.01


In [115]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.70      0.71        69
           1       0.82      0.84      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[48 21]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.64      0.69        69
           1       0.79      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[44 25]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.106+/-0.0,0.999+/-0.0,0.79+/-0.01


In [116]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.83      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[49 20]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.108+/-0.01,0.999+/-0.0,0.804+/-0.02


In [117]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,std=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[47 22]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.112+/-0.01,0.999+/-0.0,0.804+/-0.02


In [118]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.64      0.69        69
           1       0.79      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[44 25]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.103+/-0.01,0.999+/-0.0,0.806+/-0.02


In [119]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.70      0.72        69
           1       0.82      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.61      0.68        69
           1       0.79      0.89      0.84       111

    accuracy                           0.78       180
   macro avg       0.78      0.75      0.76       180
weighted avg       0.78      0.78      0.78       180

[[42 27]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.104+/-0.0,0.999+/-0.0,0.801+/-0.01


In [120]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],k=5,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[48 21]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.101+/-0.0,0.999+/-0.0,0.807+/-0.03


In [121]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.65      0.69        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[45 24]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.08+/-0.0,0.999+/-0.0,0.803+/-0.02


In [122]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.65      0.71        69
           1       0.80      0.89      0.85       111

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[45 24]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.08+/-0.0,0.999+/-0.0,0.814+/-0.01


In [123]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.62      0.69        69
           1       0.79      0.89      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[43 26]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.081+/-0.0,0.999+/-0.0,0.806+/-0.01


In [124]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.70      0.75        69
           1       0.83      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[ 48  21]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[48 21]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski',

,Time,Train,Test
model,0.081+/-0.0,0.999+/-0.0,0.809+/-0.01


In [125]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.83      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[49 20]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.078+/-0.0,0.999+/-0.0,0.809+/-0.02


In [126]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        69
           1       0.81      0.85      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.079+/-0.0,0.999+/-0.0,0.8+/-0.02


In [127]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.70      0.71        69
           1       0.82      0.84      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[48 21]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.61      0.67        69
           1       0.78      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.74       180
weighted avg       0.76      0.77      0.76       180

[[42 27]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.078+/-0.0,0.999+/-0.0,0.796+/-0.02


In [128]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.076+/-0.0,0.999+/-0.0,0.804+/-0.01


In [129]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.08+/-0.01,0.999+/-0.0,0.812+/-0.01


In [130]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'al

,Time,Train,Test
model,0.076+/-0.0,0.999+/-0.0,0.811+/-0.01


In [131]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.61      0.69        69
           1       0.79      0.90      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 42  27]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski',

,Time,Train,Test
model,0.075+/-0.0,0.999+/-0.0,0.809+/-0.02


In [132]:
df = pd.DataFrame(columns=['Time','Train','Test'], index=['model'])
show_results(df,[model],method='stat',remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space])

----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.70      0.75        69
           1       0.83      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[ 48  21]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.68      0.73        69
           1       0.82      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski',

,Time,Train,Test
model,0.078+/-0.0,0.999+/-0.0,0.811+/-0.01


In [50]:
data_ = data

In [52]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.70      0.68      0.69        69
           1       0.81      0.82      0.81       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[47 22]
 [20 91]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cos

              precision    recall  f1-score   support

           0       0.76      0.79      0.77        70
           1       0.86      0.85      0.85       110

    accuracy                           0.82       180
   macro avg       0.81      0.82      0.81       180
weighted avg       0.82      0.82      0.82       180

[[55 15]
 [17 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.85      0.57      0.68        70
           1       0.77      0.94      0.85       110

    accuracy                           0.79       180
   macro avg       0.81      0.75      0.77       180
weighted avg       0.80      0.79      0.78       180

[[ 40  30]
 [  7 103]]
             Time        Train          Test
model  0.1+/-0.01  0.999+/-0.0  0.801+/-0.02
Wife/Husband or children infected
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 

              precision    recall  f1-score   support

           0       0.77      0.62      0.69        69
           1       0.79      0.88      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.75      0.76       180
weighted avg       0.78      0.78      0.78       180

[[43 26]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.76      0.77        70
   

              precision    recall  f1-score   support

           0       0.85      0.57      0.68        70
           1       0.77      0.94      0.85       110

    accuracy                           0.79       180
   macro avg       0.81      0.75      0.77       180
weighted avg       0.80      0.79      0.78       180

[[ 40  30]
 [  7 103]]
               Time        Train          Test
model  0.097+/-0.01  0.999+/-0.0  0.801+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkow

In [54]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],method='stat',scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '


[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.60      0.71        70
           1       0.79      0.94      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 42  28]
 [  7 103]]
               Time        Train          Test
model  0.112+/-0.01  0.999+/-0.0  0.809+/

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.64      0.69        69
           1       0.79      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[44 25]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              


[[ 51  19]
 [  9 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.90      0.63      0.74        70
           1       0.80      0.95      0.87       110

    accuracy                           0.83       180
   macro avg       0.85      0.79      0.81       180
weighted avg       0.84      0.83      0.82       180

[[ 44  26]
 [  5 105]]
               Time        Train          Test
model  0.104+/-0.01  0.999+/-0.0  0.818+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0

In [56]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.70      0.72        69
           1       0.82      0.85      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.65      0.70        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[45 24]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.73      0.75        70
           1       0.83      0.86      0.85       110

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[51 19]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.59      0.69        70
           1       0.78      0.93      0.85       110

    accuracy                           0.79       180
   macro avg       0.81      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 41  29]
 [  8 102]]
               Time        Train          Test
model  0.126+/-0.01  0.999+/-0.0  0.798+/-0.01
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.59      0.66        69
           1       0.78      0.87      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.73      0.74       180
weighted avg       0.76      0.77      0.76       180

[[41 28]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.67      0.73        69
           1       0.81      0.90      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.78      0.79       180
weighted avg       0.81      0.81      0.81       180

[[ 46  23]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.60      0.67        70
           1       0.77      0.87      0.82       110

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.74       180
weighted avg       0.76      0.77      0.76       180

[[42 28]
 [14 96]]
             Time        Train          Test
model  0.12+/-0.0  0.999+/-0.0  0.778+/-0.03
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[

In [57]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.72      0.72        69
           1       0.83      0.83      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[50 19]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.73      0.64      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[44 25]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.83      0.81      0.82        70
           1       0.88      0.89      0.89       110

    accuracy                           0.86       180
   macro avg       0.85      0.85      0.85       180
weighted avg       0.86      0.86      0.86       180

[[57 13]
 [12 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.61      0.69        70
           1       0.79      0.90      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[43 27]
 [11 99]]
               Time        Train          Test
model  0.118+/-0.01  0.999+/-0.0  0.816+/-0.03
Wife/Husband or childr

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.68      0.75        69
           1       0.82      0.91      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[ 47  22]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.60      0.68        70
           1       0.78      0.90      0.84       110

    accuracy                           0.78       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.78      0.78      0.78       180

[[42 28]
 [11 99]]
               Time        Train          Test
model  0.116+/-0.01  0.999+/-0.0  0.816+/-0.03
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.68      0.74        69
           1       0.82      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.79      0.80       180
weighted avg       0.82      0.82      0.81     

In [58]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.70      0.71        69
           1       0.82      0.84      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[48 21]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.67      0.68      0.68        69
           1       0.80      0.79      0.80       111

    accuracy                           0.75       180
   macro avg       0.74      0.74      0.74       180
weighted avg       0.75      0.75      0.75       180

[[47 22]
 [23 88]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.60      0.71        70
           1       0.79      0.94      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 42  28]
 [  7 103]]
               Time        Train          Test
model  0.096+/-0.01  0.999+/-0.0  0.804+/-0.01
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.61      0.69        70
           1       0.79      0.90      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[43 27]
 [11 99]]
              Time        Train          Test
model  0.089+/-0.0  0.999+/-0.0  0.803+/-0.01
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180


In [59]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],k=3,remove=True,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        69
           1       0.83      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.79       180
weighted avg       0.80      0.80      0.80       180

[[49 20]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.68      0.70        69
           1       0.81      0.84      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.73      0.75        70
           1       0.83      0.86      0.85       110

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[51 19]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.57      0.67        70
           1       0.77      0.92      0.84       110

    accuracy                           0.78       180
   macro avg       0.79      0.74      0.76       180
weighted avg       0.79      0.78      0.77       180

[[ 40  30]
 [  9 101]]
               Time        Train        Test
model  0.123+/-0.01  0.999+/-0.0  0.8+/-0.01
Wife/Husband or ch

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.67      0.69        69
           1       0.80      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.76       180
weighted avg       0.77      0.77      0.77       180

[[46 23]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.83      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[49 20]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.60      0.69        70
           1       0.78      0.91      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 42  28]
 [ 10 100]]
               Time        Train          Test
model  0.122+/-0.01  0.999+/-0.0  0.798+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        69
           1       0.83      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.79       180
weighted avg       0.80      0.80      0.80 

In [60]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.68      0.70        69
           1       0.81      0.84      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.67      0.71      0.69        69
           1       0.81      0.78      0.80       111

    accuracy                           0.76       180
   macro avg       0.74      0.75      0.74       180
weighted avg       0.76      0.76      0.76       180

[[49 20]
 [24 87]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        70
           1       0.84      0.88      0.86       110

    accuracy                           0.82       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[51 19]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.88      0.61      0.72        70
           1       0.79      0.95      0.86       110

    accuracy                           0.82       180
   macro avg       0.84      0.78      0.79       180
weighted avg       0.83      0.82      0.81       180

[[ 43  27]
 [  6 104]]
              Time        Train         Test
model  0.115+/-0.0  0.999+/-0.0  0.81+/-0.01
Wife/Husband or ch

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.59      0.66        69
           1       0.77      0.86      0.82       111

    accuracy                           0.76       180
   macro avg       0.75      0.73      0.74       180
weighted avg       0.76      0.76      0.76       180

[[41 28]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.66      0.70        70
           1       0.80      0.85      0.82       110

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.77       180

[[46 24]
 [16 94]]
               Time        Train          Test
model  0.117+/-0.01  0.999+/-0.0  0.774+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        69
           1       0.81      0.85      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       18

In [61]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.75      0.74        69
           1       0.84      0.82      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.79      0.78       180
weighted avg       0.80      0.79      0.80       180

[[52 17]
 [20 91]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.70      0.71        69
           1       0.82      0.84      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[48 21]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.63      0.69        70
           1       0.79      0.87      0.83       110

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[44 26]
 [14 96]]
              Time        Train          Test
model  0.12+/-0.01  0.999+/-0.0  0.804+/-0.02
Wife/Husband or chil

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.70      0.72        69
           1       0.82      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              prec

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.66      0.71        70
           1       0.80      0.87      0.83       110

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 24]
 [14 96]]
               Time        Train          Test
model  0.129+/-0.01  0.999+/-0.0  0.814+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.72      0.76        69
           1       0.84      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82     

In [62]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.70      0.72        69
           1       0.82      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.68      0.67      0.67        69
           1       0.79      0.80      0.80       111

    accuracy                           0.75       180
   macro avg       0.74      0.73      0.73       180
weighted avg       0.75      0.75      0.75       180

[[46 23]
 [22 89]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.79      0.77        70
           1       0.86      0.85      0.85       110

    accuracy                           0.82       180
   macro avg       0.81      0.82      0.81       180
weighted avg       0.82      0.82      0.82       180

[[55 15]
 [17 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.57      0.68        70
           1       0.77      0.93      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.75      0.76       180
weighted avg       0.80      0.79      0.78       180

[[ 40  30]
 [  8 102]]
               Time        Train          Test
model  0.119+/-0.04  0.999+/-0.0  0.803+/-0.01
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.65      0.70        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[45 24]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.60      0.70        70
           1       0.78      0.93      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.76      0.78       180
weighted avg       0.81      0.80      0.79       180

[[ 42  28]
 [  8 102]]
               Time        Train          Test
model  0.123+/-0.05  0.999+/-0.0  0.812+/-0.01
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82 

In [63]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.70      0.72        69
           1       0.82      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.70      0.77      0.73        70
           1       0.84      0.79      0.82       110

    accuracy                           0.78       180
   macro avg       0.77      0.78      0.78       180
weighted avg       0.79      0.78      0.78       180

[[54 16]
 [23 87]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.60      0.71        70
           1       0.79      0.94      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 42  28]
 [  7 103]]
               Time        Train        Test
model  0.106+/-0.01  0.999+/-0.0  0.8+/-0.01
Wife/Husband or childr

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.65      0.70        69
           1       0.80      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.63      0.73        70
           1       0.80      0.94      0.86       110

    accuracy                           0.82       180
   macro avg       0.83      0.78      0.79       180
weighted avg       0.82      0.82      0.81       180

[[ 44  26]
 [  7 103]]
              Time        Train          Test
model  0.099+/-0.0  0.999+/-0.0  0.813+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81   

In [64]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        69
           1       0.81      0.85      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.65      0.70        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[45 24]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        70
           1       0.84      0.88      0.86       110

    accuracy                           0.82       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[51 19]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.63      0.73        70
           1       0.80      0.94      0.86       110

    accuracy                           0.82       180
   macro avg       0.83      0.78      0.79       180
weighted avg       0.82      0.82      0.81       180

[[ 44  26]
 [  7 103]]
               Time        Train          Test
model  0.125+/-0.04  0.999+/-0.0  0.803+/-0.02
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.59      0.67        69
           1       0.78      0.88      0.83       111

    accuracy                           0.77       180
   macro avg       0.77      0.74      0.75       180
weighted avg       0.77      0.77      0.77       180

[[41 28]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.61      0.70        69
           1       0.79      0.92      0.85       111

    accuracy                           0.80       180
   macro avg       0.81      0.76      0.77       180
weighted avg       0.80      0.80      0.79       180

[[ 42  27]
 [  9 102]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.54      0.66        70
           1       0.76      0.93      0.84       110

    accuracy                           0.78       180
   macro avg       0.79      0.74      0.75       180
weighted avg       0.79      0.78      0.77       180

[[ 38  32]
 [  8 102]]
               Time        Train          Test
model  0.107+/-0.01  0.999+/-0.0  0.792+/-0.03
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80 

In [66]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent']:
        data = data_.drop(col,1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.71      0.72        69
           1       0.82      0.84      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.63      0.70        70
           1       0.79      0.89      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[44 26]
 [12 98]]
               Time        Train         Test
model  0.128+/-0.01  0.999+/-0.0  0.81+/-0.02
Wife/Husband or chil

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.72      0.75        69
           1       0.84      0.87      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[50 19]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.64      0.71        70
           1       0.80      0.89      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 25]
 [12 98]]
              Time        Train          Test
model  0.131+/-0.0  0.999+/-0.0  0.817+/-0.02
Is_young
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.71      0.76        69
           1       0.83      0.90      0.87       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       

In [67]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year']:
        data = data_.drop([col,'Birthday_year'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.74      0.74        69
           1       0.84      0.85      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[51 18]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.73      0.64      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[44 25]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.80      0.79        70
           1       0.87      0.86      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[56 14]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.61      0.70        70
           1       0.79      0.91      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 43  27]
 [ 10 100]]
            Time        Train          Test
model  0.1+/-0.0  0.996+/-0.0  0.812+/-0.02
Medical_Expenses_Fam


[[45 24]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.68      0.73        69
           1       0.82      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.80      0.80        70
           1       0.87      0.87      0.87       110

    accuracy                           0.84       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.84      0.84      0.84       180

[[56 14]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_t

In [68]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year']:
        data = data_.drop([col,'Birthday_year'],1)
        print(col)
        print(show_results(df,[model],method='stat',scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.74      0.74        69
           1       0.84      0.85      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[51 18]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.65      0.68        69
           1       0.79      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.77      0.76       180

[[45 24]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

              precision    recall  f1-score   support

           0       0.78      0.77      0.78        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.66      0.73        70
           1       0.81      0.91      0.85       110

    accuracy                           0.81       180
   macro avg       0.81      0.78      0.79       180
weighted avg       0.81      0.81      0.81       180

[[ 46  24]
 [ 10 100]]
               Time        Train          Test
model  0.095+/-0.01  0.996+/-0.0  0.822+/-0.01
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'm

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.71      0.77        69
           1       0.83      0.91      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.81      0.82       180
weighted avg       0.83      0.83      0.83       180

[[ 49  20]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.61      0.71        70
           1       0.79      0.93      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 43  27]
 [  8 102]]
              Time        Train          Test
model  0.106+/-0.0  0.996+/-0.0  0.817+/-0.01


In [69]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family']:
        data = data_.drop([col,'Medical_Expenses_Family'],1)
        print(col)
        print(show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.72      0.72        69
           1       0.83      0.83      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[50 19]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.65      0.68        69
           1       0.79      0.83      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.74      0.74       180
weighted avg       0.76      0.76      0.76       180

[[45 24]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.74      0.76        70
           1       0.84      0.87      0.86       110

    accuracy                           0.82       180
   macro avg       0.81      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 18]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.91      0.56      0.69        70
           1       0.77      0.96      0.86       110

    accuracy                           0.81       180
   macro avg       0.84      0.76      0.77       180
weighted avg       0.83      0.81      0.79       180

[[ 39  31]
 [  4 106]]
              Time        Train          Test
model  0.127+/-0.0  0.999+/-0.0  0.812+/-0.01
Wife/Husband or 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.64      0.72        69
           1       0.80      0.91      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 44  25]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.71      0.77        69
           1       0.83      0.91      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.81      0.82       180
weighted avg       0.83      0.83      0.83       180

[[ 49  20]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
      

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.89      0.57      0.70        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.76      0.78       180
weighted avg       0.82      0.81      0.79       180

[[ 40  30]
 [  5 105]]
              Time        Train          Test
model  0.125+/-0.0  0.999+/-0.0  0.814+/-0.01


In [70]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Infected_per_Family']:
        data = data_.drop([col,'Infected_per_Family'],1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        69
           1       0.82      0.85      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.65      0.69        69
           1       0.80      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[45 24]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.77      0.78        70
           1       0.86      0.87      0.86       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
              Time        Train          Test
model  0.128+/-0.0  0.999+/-0.0  0.809+/-0.01
Wife/Husband or 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.68      0.74        69
           1       0.82      0.90      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[ 47  22]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.68      0.73        69
           1       0.82      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.60      0.69        70
           1       0.78      0.92      0.85       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 42  28]
 [  9 101]]
               Time        Train          Test
model  0.127+/-0.01  0.999+/-0.0  0.813+/-0.02


In [71]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Birthday_year']:
        data = data_.drop([col,'Birthday_year'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.77      0.75        69
           1       0.85      0.83      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[53 16]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.69      0.68      0.69        69
           1       0.80      0.81      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.76      0.76       180

[[47 22]
 [21 90]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.80      0.79        70
           1       0.87      0.86      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[56 14]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.66      0.72        70
           1       0.80      0.89      0.84       110

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 24]
 [12 98]]
             Time        Train          Test
model  0.1+/-0.01  0.996+/-0.0  0.817+/-0.01
Medical_Expenses_Famil

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.64      0.69        69
           1       0.80      0.87      0.83       111

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[44 25]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.64      0.70        70
           1       0.79      0.87      0.83       110

    accuracy                           0.78       180
   macro avg       0.78      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[45 25]
 [14 96]]
               Time        Train         Test
model  0.123+/-0.04  0.996+/-0.0  0.81+/-0.02


In [72]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Birthday_year']:
        data = data_.drop([col, 'Birthday_year'],1)
        print(col)
        print(show_results(df,[model],k=3,remove=True,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.75      0.73        69
           1       0.84      0.81      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[52 17]
 [21 90]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.68      0.68      0.68        69
           1       0.80      0.80      0.80       111

    accuracy                           0.76       180
   macro avg       0.74      0.74      0.74       180
weighted avg       0.76      0.76      0.76       180

[[47 22]
 [22 89]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.77      0.77        70
           1       0.85      0.85      0.85       110

    accuracy                           0.82       180
   macro avg       0.81      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[54 16]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.64      0.71        70
           1       0.80      0.90      0.85       110

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[45 25]
 [11 99]]
               Time        Train          Test
model  0.123+/-0.01  0.996+/-0.0  0.811+/-0.01
Medical_Expenses_F

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        69
           1       0.84      0.89      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       180

[[50 19]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
               Time        Train          Test
model  0.125+/-0.01  0.996+/-0.0  0.811+/-0.02


In [73]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Parents or siblings infected']:
        data = data_.drop([col, 'Parents or siblings infected'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        69
           1       0.80      0.85      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.72      0.72        69
           1       0.83      0.82      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[50 19]
 [20 91]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.75        70
           1       0.82      0.89      0.86       110

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 21]
 [12 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.88      0.60      0.71        70
           1       0.79      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.77      0.79       180
weighted avg       0.82      0.81      0.80       180

[[ 42  28]
 [  6 104]]
              Time        Train          Test
model  0.124+/-0.0  0.999+/-0.0  0.802+/-0.02
Medical_Expenses

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.58      0.67        69
           1       0.78      0.91      0.84       111

    accuracy                           0.78       180
   macro avg       0.79      0.74      0.76       180
weighted avg       0.79      0.78      0.77       180

[[ 40  29]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.67      0.73        69
           1       0.81      0.90      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.78      0.79       180
weighted avg       0.81      0.81      0.81       180

[[ 46  23]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
      

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.88      0.61      0.72        70
           1       0.79      0.95      0.86       110

    accuracy                           0.82       180
   macro avg       0.84      0.78      0.79       180
weighted avg       0.83      0.82      0.81       180

[[ 43  27]
 [  6 104]]
              Time        Train          Test
model  0.121+/-0.0  0.999+/-0.0  0.809+/-0.01


In [74]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young']:
        data = data_.drop([col,'Is_young'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.77      0.76        69
           1       0.85      0.85      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[53 16]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        69
           1       0.82      0.85      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.79      0.79        70
           1       0.86      0.87      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[55 15]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.63      0.70        70
           1       0.79      0.90      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[44 26]
 [11 99]]
              Time        Train          Test
model  0.122+/-0.0  0.999+/-0.0  0.811+/-0.02
Wife/Husband or chil

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.63      0.70        70
           1       0.79      0.89      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[44 26]
 [12 98]]
               Time        Train          Test
model  0.124+/-0.01  0.999+/-0.0  0.816+/-0.03


In [75]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young']:
        data = data_.drop([col,'Is_young'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.70      0.72        69
           1       0.82      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.57      0.68        70
           1       0.77      0.93      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.75      0.76       180
weighted avg       0.80      0.79      0.78       180

[[ 40  30]
 [  8 102]]
              Time        Train          Test
model  0.099+/-0.0  0.999+/-0.0  0.803+/-0.01
Wife/Husband or 

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.80      0.79        70
           1       0.87      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.83      0.83       180
weighted avg       0.83      0.83      0.83       180

[[56 14]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.59      0.69        70
           1       0.78      0.93      0.85       110

    accuracy                           0.79       180
   macro avg       0.81      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 41  29]
 [  8 102]]
               Time        Train          Test
model  0.105+/-0.01  0.999+/-0.0  0.811+/-0.01


In [76]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Severity']:
        data = data_.drop([col,'Severity'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.75      0.76        69
           1       0.85      0.86      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 17]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '


[[51 18]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.77      0.78        70
           1       0.86      0.87      0.86       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.61      0.72        70
           1       0.79      0.94      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.78      0.79       180
weighted avg       0.82      0.81      0.80       180

[[ 43  27]
 [  7 103]]
               Time        Train          Test
model  0.096+/-0.01  0.999+/-0.0  0.816+/

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.72      0.74        69
           1       0.83      0.85      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.80      0.80      0.80       180

[[50 19]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.74      0.76        69
           1       0.84      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[51 18]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.87      0.66      0.75        70
           1       0.81      0.94      0.87       110

    accuracy                           0.83       180
   macro avg       0.84      0.80      0.81       180
weighted avg       0.83      0.83      0.82       180

[[ 46  24]
 [  7 103]]
              Time        Train          Test
model  0.12+/-0.01  0.999+/-0.0  0.823+/-0.01


In [77]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family']:
        data = data_.drop([col,'Medical_Expenses_Family'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.71      0.72        69
           1       0.82      0.83      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[49 20]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.65      0.69        69
           1       0.80      0.85      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[45 24]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

              precision    recall  f1-score   support

           0       0.78      0.74      0.76        70
           1       0.84      0.86      0.85       110

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 18]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.89      0.57      0.70        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.76      0.78       180
weighted avg       0.82      0.81      0.79       180

[[ 40  30]
 [  5 105]]
               Time        Train          Test
model  0.119+/-0.04  0.999+/-0.0  0.813+/-0.01
Wife/Husband or children infected
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbor


[[45 24]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.64      0.71        69
           1       0.80      0.90      0.85       111

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 44  25]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        69
           1       0.84      0.89      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       180

[[50 19]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.87      0.59      0.70        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.77      0.78       180
weighted avg       0.82      0.81      0.80       180

[[ 41  29]
 [  6 104]]
              Time        Train          Test
model  0.099+/-0.0  0.999+/-0.0  0.812+/-0.01


In [78]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young']:
        data = data_.drop([col,'Is_young'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.72      0.75        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[50 19]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.71      0.75        69
           1       0.83      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[49 20]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.63      0.72        70
           1       0.80      0.92      0.85       110

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 44  26]
 [  9 101]]
              Time        Train          Test
model  0.127+/-0.0  0.999+/-0.0  0.817+/-0.01
Wife/Husband or 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.63      0.71        70
           1       0.79      0.91      0.85       110

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 44  26]
 [ 10 100]]
               Time        Train          Test
model  0.148+/-0.02  0.999+/-0.0  0.818+/-0.02


In [79]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status']:
        data = data_.drop([col,'Birthday_year','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.74      0.74        69
           1       0.84      0.85      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[51 18]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.64      0.68        69
           1       0.79      0.85      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.75       180
weighted avg       0.76      0.77      0.76       180

[[44 25]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.61      0.70        70
           1       0.79      0.91      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 43  27]
 [ 10 100]]
              Time        Train          Test
model  0.089+/-0.0  0.996+/-0.0  0.816+/-0.02
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.72      0.76        69
           1       0.84      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.80      0.81       180
weighted avg       0.82      0.

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.65      0.73        69
           1       0.81      0.91      0.86       111

    accuracy                           0.81       180
   macro avg       0.81      0.78      0.79       180
weighted avg       0.81      0.81      0.81       180

[[ 45  24]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.73      0.76        70
           1       0.83      0.87      0.85       110

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[51 19]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

In [80]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status']:
        data = data_.drop([col,'Birthday_year','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],method='stat',scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        69
           1       0.85      0.85      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[52 17]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.64      0.67        69
           1       0.79      0.83      0.81       111

    accuracy                           0.76       180
   macro avg       0.74      0.73      0.74       180
weighted avg       0.75      0.76      0.75       180

[[44 25]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.64      0.73        70
           1       0.80      0.92      0.86       110

    accuracy                           0.81       180
   macro avg       0.82      0.78      0.79       180
weighted avg       0.81      0.81      0.81       180

[[ 45  25]
 [  9 101]]
             Time        Train          Test
model  0.1+/-0.01  0.996+/-0.0  0.822+/-0.01
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82

              precision    recall  f1-score   support

           0       0.78      0.65      0.71        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.67      0.74        70
           1       0.81      0.91      0.86       110

    accuracy                           0.82       180
   macro avg       0.82      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[ 47  23]
 [ 10 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.61      0.69        70

In [81]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family', 'City']:
        data = data_.drop([col,'Medical_Expenses_Family','Is_young'],1)
        print(col)
        print(show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.71      0.72        69
           1       0.82      0.83      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[49 20]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.65      0.68        69
           1       0.79      0.83      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.74      0.74       180
weighted avg       0.76      0.76      0.76       180

[[45 24]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.74      0.76        70
           1       0.84      0.87      0.86       110

    accuracy                           0.82       180
   macro avg       0.81      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 18]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.89      0.57      0.70        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.76      0.78       180
weighted avg       0.82      0.81      0.79       180

[[ 40  30]
 [  5 105]]
               Time        Train          Test
model  0.126+/-0.01  0.999+/-0.0  0.811+/-0.02
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.88      0.55      0.68        69
           1       0.77      0.95      0.85       111

    accuracy                           0.80       180
   macro avg       0.83      0.75      0.77       180
weighted avg       0.82      0.80      0.79       180

[[ 38  31]
 [  5 106]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.62      0.71        69
           1       0.80      0.92      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 43  26]
 [  9 102]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
      

In [82]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Infected_per_Family','Is_young']:
        data = data_.drop([col,'Infected_per_Family','Is_young'],1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.77      0.79        70
           1       0.86      0.88      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[54 16]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.88      0.60      0.71        70
           1       0.79      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.77      0.79       180
weighted avg       0.82      0.81      0.80       180

[[ 42  28]
 [  6 104]]
               Time        Train          Test
model  0.123+/-0.01  0.999+/-0.0  0.812+/-0.01
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.68      0.73        69
           1       0.82      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [83]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Birthday_year','Wife/Husband or children infected']:
        data = data_.drop([col,'Birthday_year','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.77      0.75        69
           1       0.85      0.83      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[53 16]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.65      0.68        69
           1       0.79      0.83      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.74      0.74       180
weighted avg       0.76      0.76      0.76       180

[[45 24]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.77      0.79        70
           1       0.86      0.88      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[54 16]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.61      0.71        70
           1       0.79      0.93      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 43  27]
 [  8 102]]
              Time        Train          Test
model  0.084+/-0.0  0.997+/-0.0  0.803+/-0.02
City
-----------

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.64      0.69        69
           1       0.79      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[44 25]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.83      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[49 20]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [84]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Birthday_year','Marital_Status']:
        data = data_.drop([col, 'Birthday_year','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],k=3,remove=True,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.74      0.72        69
           1       0.83      0.81      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.79      0.78      0.78       180

[[51 18]
 [21 90]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.67      0.69        69
           1       0.80      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.76       180
weighted avg       0.77      0.77      0.77       180

[[46 23]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.80      0.79        70
           1       0.87      0.86      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[56 14]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.69      0.74        70
           1       0.82      0.89      0.85       110

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[48 22]
 [12 98]]
              Time        Train          Test
model  0.12+/-0.01  0.996+/-0.0  0.819+/-0.01
Medical_Expenses_Fam

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.62      0.67        69
           1       0.78      0.85      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.74      0.74       180
weighted avg       0.76      0.76      0.76       180

[[43 26]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        69
           1       0.81      0.85      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [85]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Parents or siblings infected','Is_young']:
        data = data_.drop([col, 'Parents or siblings infected','Is_young'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.68      0.70        69
           1       0.81      0.83      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.76       180
weighted avg       0.77      0.77      0.77       180

[[47 22]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.68      0.70        69
           1       0.81      0.84      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[47 22]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.73      0.75        70
           1       0.83      0.86      0.85       110

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[51 19]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.89      0.59      0.71        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.84      0.77      0.78       180
weighted avg       0.83      0.81      0.80       180

[[ 41  29]
 [  5 105]]
              Time        Train          Test
model  0.123+/-0.0  0.999+/-0.0  0.804+/-0.01
Medical_Expenses

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.58      0.67        69
           1       0.78      0.91      0.84       111

    accuracy                           0.78       180
   macro avg       0.79      0.74      0.76       180
weighted avg       0.79      0.78      0.77       180

[[ 40  29]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.67      0.73        69
           1       0.81      0.90      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.78      0.79       180
weighted avg       0.81      0.81      0.81       180

[[ 46  23]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
      

In [86]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','City']:
        data = data_.drop([col,'Is_young','Title'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        69
           1       0.85      0.85      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[52 17]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        69
           1       0.82      0.85      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.80      0.82        70
           1       0.88      0.90      0.89       110

    accuracy                           0.86       180
   macro avg       0.86      0.85      0.85       180
weighted avg       0.86      0.86      0.86       180

[[56 14]
 [11 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
               Time        Train          Test
model  0.116+/-0.01  0.999+/-0.0  0.817+/-0.02
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.65      0.71        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[45 24]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.74      0.75        69
           1       0.84      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[51 18]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [87]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Birthday_year']:
        data = data_.drop([col,'Is_young','Birthday_year'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.72      0.74        69
           1       0.83      0.85      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.80      0.80      0.80       180

[[50 19]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.62      0.66        69
           1       0.78      0.84      0.81       111

    accuracy                           0.76       180
   macro avg       0.74      0.73      0.74       180
weighted avg       0.75      0.76      0.75       180

[[43 26]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.80      0.79        70
           1       0.87      0.86      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[56 14]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.60      0.68        70
           1       0.78      0.90      0.84       110

    accuracy                           0.78       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.78      0.78      0.78       180

[[42 28]
 [11 99]]
               Time        Train          Test
model  0.103+/-0.01  0.996+/-0.0  0.814+/-0.02
Medical_Expenses_F

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [88]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Severity','Is_young']:
        data = data_.drop([col,'Severity','Is_young'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.75      0.76        69
           1       0.85      0.86      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.81      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 17]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.77      0.78        70
           1       0.86      0.87      0.86       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.86      0.61      0.72        70
           1       0.79      0.94      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.78      0.79       180
weighted avg       0.82      0.81      0.80       180

[[ 43  27]
 [  7 103]]
               Time        Train          Test
model  0.183+/-0.02  0.999+/-0.0  0.816+/-0.01
Medical_Expens

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.72      0.74        69
           1       0.83      0.85      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.80      0.80      0.80       180

[[50 19]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.74      0.76        69
           1       0.84      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[51 18]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [89]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family','Wife/Husband or children infected']:
        data = data_.drop([col,'Medical_Expenses_Family','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        69
           1       0.83      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.79       180
weighted avg       0.80      0.80      0.80       180

[[49 20]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.67      0.68        69
           1       0.80      0.82      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.74      0.75       180
weighted avg       0.76      0.76      0.76       180

[[46 23]
 [20 91]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.77      0.78        70
           1       0.86      0.87      0.86       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.85      0.57      0.68        70
           1       0.77      0.94      0.85       110

    accuracy                           0.79       180
   macro avg       0.81      0.75      0.77       180
weighted avg       0.80      0.79      0.78       180

[[ 40  30]
 [  7 103]]
               Time        Train         Test
model  0.187+/-0.02  0.999+/-0.0  0.81+/-0.02
City
-----------

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.51      0.61        69
           1       0.75      0.91      0.82       111

    accuracy                           0.76       180
   macro avg       0.76      0.71      0.72       180
weighted avg       0.76      0.76      0.74       180

[[ 35  34]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.59      0.68        69
           1       0.78      0.91      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 41  28]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
      

In [90]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Title']:
        data = data_.drop([col,'Is_young','Title'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.79      0.80        70
           1       0.87      0.88      0.87       110

    accuracy                           0.84       180
   macro avg       0.84      0.83      0.84       180
weighted avg       0.84      0.84      0.84       180

[[55 15]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.61      0.70        70
           1       0.79      0.91      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 43  27]
 [ 10 100]]
               Time        Train          Test
model  0.181+/-0.02  0.999+/-0.0  0.817+/-0.02
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [91]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status','Is_young']:
        data = data_.drop([col,'Birthday_year','Marital_Status','Is_young'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.72      0.74        69
           1       0.83      0.86      0.84       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[50 19]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.62      0.68        69
           1       0.79      0.87      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[43 26]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.60      0.69        70
           1       0.78      0.92      0.85       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[ 42  28]
 [  9 101]]
               Time        Train          Test
model  0.092+/-0.01  0.996+/-0.0  0.816+/-0.02
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.74      0.77        69
           1       0.84      0.88      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.82       180
weighted avg       0.83      

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.67      0.74        69
           1       0.81      0.91      0.86       111

    accuracy                           0.82       180
   macro avg       0.82      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[ 46  23]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.74      0.78        70
           1       0.84      0.89      0.87       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[52 18]
 [12 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

In [92]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status','Wife/Husband or children infected']:
        data = data_.drop([col,'Birthday_year','Marital_Status','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],method='stat',scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        69
           1       0.85      0.85      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[52 17]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.65      0.68        69
           1       0.79      0.83      0.81       111

    accuracy                           0.76       180
   macro avg       0.75      0.74      0.74       180
weighted avg       0.76      0.76      0.76       180

[[45 24]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

              precision    recall  f1-score   support

           0       0.79      0.77      0.78        70
           1       0.86      0.87      0.86       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.60      0.70        70
           1       0.78      0.93      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.76      0.78       180
weighted avg       0.81      0.80      0.79       180

[[ 42  28]
 [  8 102]]
              Time        Train          Test
model  0.09+/-0.01  0.996+/-0.0  0.807+/-0.02
City
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tr

In [93]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family', 'City','Severity']:
        data = data_.drop([col,'Medical_Expenses_Family','Is_young','Severity'],1)
        print(col)
        print(show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        69
           1       0.81      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.77       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.71      0.65      0.68        69
           1       0.79      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.77      0.76       180

[[45 24]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine'

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.80      0.79        70
           1       0.87      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.83      0.83       180
weighted avg       0.83      0.83      0.83       180

[[56 14]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.85      0.66      0.74        70
           1       0.81      0.93      0.86       110

    accuracy                           0.82       180
   macro avg       0.83      0.79      0.80       180
weighted avg       0.83      0.82      0.82       180

[[ 46  24]
 [  8 102]]
              Time        Train          Test
model  0.137+/-0.0  0.999+/-0.0  0.818+/-0.01
Infected_per_Fam


[[50 19]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.65      0.68        69
           1       0.79      0.84      0.82       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.77      0.76       180

[[45 24]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.72      0.75        69
           1       0.84      0.87      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[50 19]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_t

In [94]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Infected_per_Family','Is_young','City']:
        data = data_.drop([col,'Infected_per_Family','Is_young','City'],1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.71      0.72        69
           1       0.82      0.83      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.78      0.78      0.78       180

[[49 20]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        69
           1       0.80      0.85      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[46 23]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cos

{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.76      0.76      0.76        70
           1       0.85      0.85      0.85       110

    accuracy                           0.81       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.81      0.81      0.81       180

[[53 17]
 [17 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.85      0.59      0.69        70
           1       0.78      0.94      0.85       110

    accuracy                           0.80       180
   macro avg       0.82      0.76      0.77       180
weighted avg       0.81      0.80      0.79       180

[[ 41  29]
 [  7 103]]
               Time        Train         Test
model  0.124+/-0.01  0.999+/-0.0  0.806+/-0.0
Wife/Husband or chil

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [95]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Birthday_year','Wife/Husband or children infected','Is_young']:
        data = data_.drop([col,'Birthday_year','Wife/Husband or children infected','Is_young'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.74      0.74        69
           1       0.84      0.85      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[51 18]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.61      0.67        69
           1       0.78      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.74       180
weighted avg       0.76      0.77      0.76       180

[[42 27]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.76      0.78        70
           1       0.85      0.88      0.87       110

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[53 17]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
              Time        Train          Test
model  0.097+/-0.0  0.997+/-0.0  0.799+/-0.03
City
-----------

              precision    recall  f1-score   support

           0       0.76      0.68      0.72        69
           1       0.81      0.86      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.62      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.74      0.75       180
weighted avg       0.77      0.77      0.77       180

[[43 26]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
   

In [96]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Birthday_year','Marital_Status','Wife/Husband or children infected']:
        data = data_.drop([col, 'Birthday_year','Marital_Status','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],k=3,remove=True,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.75      0.75        69
           1       0.85      0.84      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.80      0.79       180
weighted avg       0.81      0.81      0.81       180

[[52 17]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.71      0.67      0.69        69
           1       0.80      0.83      0.81       111

    accuracy                           0.77       180
   macro avg       0.75      0.75      0.75       180
weighted avg       0.76      0.77      0.77       180

[[46 23]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.60      0.70        70
           1       0.78      0.93      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.76      0.78       180
weighted avg       0.81      0.80      0.79       180

[[ 42  28]
 [  8 102]]
               Time        Train          Test
model  0.123+/-0.01  0.997+/-0.0  0.803+/-0.01
City
---------

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        69
           1       0.83      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[49 20]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [97]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Parents or siblings infected','Is_young','Medical_Expenses_Family']:
        data = data_.drop([col, 'Parents or siblings infected','Is_young','Medical_Expenses_Family'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.70      0.72        69
           1       0.82      0.85      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.69      0.62      0.66        69
           1       0.78      0.83      0.80       111

    accuracy                           0.75       180
   macro avg       0.74      0.73      0.73       180
weighted avg       0.75      0.75      0.75       180

[[43 26]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowsk

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.89      0.59      0.71        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.84      0.77      0.78       180
weighted avg       0.83      0.81      0.80       180

[[ 41  29]
 [  5 105]]
               Time        Train          Test
model  0.125+/-0.01  0.999+/-0.0  0.814+/-0.01
City
---------

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.55      0.64        69
           1       0.76      0.89      0.82       111

    accuracy                           0.76       180
   macro avg       0.76      0.72      0.73       180
weighted avg       0.76      0.76      0.75       180

[[38 31]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.59      0.68        69
           1       0.78      0.91      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 41  28]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
          

In [98]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','City','Marital_Status']:
        data = data_.drop([col,'Is_young','Title','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.74      0.76        69
           1       0.84      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[51 18]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[48 21]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.80      0.81        70
           1       0.87      0.88      0.88       110

    accuracy                           0.85       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.85      0.85      0.85       180

[[56 14]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
               Time        Train          Test
model  0.127+/-0.01  0.999+/-0.0  0.814+/-0.02
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        69
           1       0.78      0.81      0.80       111

    accuracy                           0.74       180
   macro avg       0.73      0.72      0.73       180
weighted avg       0.74      0.74      0.74       180

[[44 25]
 [21 90]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.57      0.62        69
           1       0.76      0.85      0.80       111

    accuracy                           0.74       180
   macro avg       0.73      0.71      0.71       180
weighted avg       0.73      0.74      0.73       180

[[39 30]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [99]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Birthday_year','Title']:
        data = data_.drop([col,'Is_young','Birthday_year','Title'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.74      0.74        69
           1       0.84      0.85      0.84       111

    accuracy                           0.81       180
   macro avg       0.79      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[51 18]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.73      0.64      0.68        69
           1       0.79      0.86      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[44 25]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

              precision    recall  f1-score   support

           0       0.79      0.80      0.79        70
           1       0.87      0.86      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[56 14]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.63      0.70        70
           1       0.79      0.90      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.76      0.77       180
weighted avg       0.80      0.79      0.79       180

[[44 26]
 [11 99]]
              Time        Train          Test
model  0.069+/-0.0  0.996+/-0.0  0.814+/-0.01
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric'

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.74      0.77        69
           1       0.84      0.88      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.82       180
weighted avg       0.83      0.83      0.83       180

[[51 18]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.79      0.79        70
           1       0.86      0.87      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[55 15]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [100]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Severity','Is_young','Marital_Status']:
        data = data_.drop([col,'Severity','Is_young','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=StandardScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.75      0.76        69
           1       0.85      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 17]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.65      0.69        69
           1       0.80      0.85      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.75       180
weighted avg       0.77      0.77      0.77       180

[[45 24]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.61      0.71        70
           1       0.79      0.93      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 43  27]
 [  8 102]]
              Time        Train          Test
model  0.067+/-0.0  0.999+/-0.0  0.812+/-0.01
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.72      0.75        69
           1       0.84      0.87      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.

In [101]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family','Wife/Husband or children infected','Severity']:
        data = data_.drop([col,'Medical_Expenses_Family','Wife/Husband or children infected','Severity'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.67      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.61      0.70      0.65        69
           1       0.79      0.72      0.75       111

    accuracy                           0.71       180
   macro avg       0.70      0.71      0.70       180
weighted avg       0.72      0.71      0.71       180

[[48 21]
 [31 80]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cos


[[51 18]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.80      0.79        70
           1       0.87      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.83      0.83       180
weighted avg       0.83      0.83      0.83       180

[[56 14]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.61      0.71        70
           1       0.79      0.93      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 43  27]
 [  8 102]]
              Time        Train          Test
model  0.065+/-0.0  0.999+/-0.0  0.812+/-0

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.70      0.72        69
           1       0.82      0.85      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[48 21]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.77      0.78        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[54 16]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [102]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Title','Birthday_year']:
        data = data_.drop([col,'Is_young','Title','Birthday_year'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.72      0.73        69
           1       0.83      0.84      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[50 19]
 [18 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.70      0.71        69
           1       0.81      0.83      0.82       111

    accuracy                           0.78       180
   macro avg       0.77      0.76      0.76       180
weighted avg       0.78      0.78      0.78       180

[[48 21]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '


[[51 18]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.79      0.81        70
           1       0.87      0.90      0.88       110

    accuracy                           0.86       180
   macro avg       0.85      0.84      0.85       180
weighted avg       0.85      0.86      0.85       180

[[55 15]
 [11 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.63      0.70        70
           1       0.79      0.89      0.84       110

    accuracy                           0.79       180
   macro avg       0.79      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[44 26]
 [12 98]]
              Time        Train          Test
model  0.092+/-0.0  0.996+/-0.0  0.822+/-0.02


              precision    recall  f1-score   support

           0       0.81      0.72      0.76        69
           1       0.84      0.89      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       180

[[50 19]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.70      0.73        69
           1       0.82      0.87      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[48 21]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.74      0.76        69
   

In [103]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status','Is_young','Title']:
        data = data_.drop([col,'Birthday_year','Marital_Status','Is_young','Title'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.82      0.74      0.78        69
           1       0.85      0.90      0.87       111

    accuracy                           0.84       180
   macro avg       0.84      0.82      0.83       180
weighted avg       0.84      0.84      0.84       180

[[ 51  18]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.77      0.64      0.70        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[44 25]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cos

              precision    recall  f1-score   support

           0       0.82      0.79      0.80        70
           1       0.87      0.89      0.88       110

    accuracy                           0.85       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.85      0.85      0.85       180

[[55 15]
 [12 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.56      0.67        70
           1       0.77      0.93      0.84       110

    accuracy                           0.78       180
   macro avg       0.80      0.74      0.75       180
weighted avg       0.79      0.78      0.77       180

[[ 39  31]
 [  8 102]]
               Time        Train          Test
model  0.087+/-0.01  0.993+/-0.0  0.817+/-0.02
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'm

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.64      0.62      0.63        69
           1       0.77      0.78      0.78       111

    accuracy                           0.72       180
   macro avg       0.71      0.70      0.70       180
weighted avg       0.72      0.72      0.72       180

[[43 26]
 [24 87]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.75      0.52      0.62        69
           1       0.75      0.89      0.81       111

    accuracy                           0.75       180
   macro avg       0.75      0.71      0.72       180
weighted avg       0.75      0.75      0.74       180

[[36 33]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [109]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family', 'City','Severity','Wife/Husband or children infected']:
        data = data_.drop([col,'Medical_Expenses_Family','City','Severity','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.77      0.72      0.75        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[50 19]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine'


[[48 21]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.73      0.74        70
           1       0.83      0.85      0.84       110

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[51 19]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.64      0.71        70
           1       0.80      0.90      0.85       110

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[45 25]
 [11 99]]
              Time        Train          Test
model  0.099+/-0.0  0.999+/-0.0  0.794+/-0.01


              precision    recall  f1-score   support

           0       0.77      0.74      0.76        69
           1       0.84      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[51 18]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.72      0.75        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[50 19]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.72      0.72      0.72        69
   

In [105]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Infected_per_Family','Is_young','City','Marital_Status']:
        data = data_.drop([col,'Infected_per_Family','Is_young','City','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.75      0.76        69
           1       0.85      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.81       180
weighted avg       0.82      0.82      0.82       180

[[52 17]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.68      0.71        69
           1       0.81      0.86      0.83       111

    accuracy                           0.79       180
   macro avg       0.78      0.77      0.77       180
weighted avg       0.79      0.79      0.79       180

[[47 22]
 [16 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cos


[[ 46  23]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.89      0.60      0.72        70
           1       0.79      0.95      0.86       110

    accuracy                           0.82       180
   macro avg       0.84      0.78      0.79       180
weighted avg       0.83      0.82      0.81       180

[[ 42  28]
 [  5 105]]
              Time        Train          Test
model  0.104+/-0.0  0.999+/-0.0  0.818+/-0


[[ 47  22]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.82      0.74      0.78        69
           1       0.85      0.90      0.87       111

    accuracy                           0.84       180
   macro avg       0.84      0.82      0.83       180
weighted avg       0.84      0.84      0.84       180

[[ 51  18]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        69
           1       0.82      0.86      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': '

In [106]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','City','Marital_Status','Title']:
        data = data_.drop([col,'Is_young','Title','Marital_Status','Title'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.74      0.76        69
           1       0.84      0.86      0.85       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.82       180

[[51 18]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       180

[[48 21]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.80      0.81        70
           1       0.87      0.88      0.88       110

    accuracy                           0.85       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.85      0.85      0.85       180

[[56 14]
 [13 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.61      0.70        70
           1       0.79      0.92      0.85       110

    accuracy                           0.80       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.80      0.80      0.79       180

[[ 43  27]
 [  9 101]]
               Time        Train          Test
model  0.115+/-0.01  0.999+/-0.0  0.814+/-0.02
Wife/Husband o

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        69
           1       0.78      0.81      0.80       111

    accuracy                           0.74       180
   macro avg       0.73      0.72      0.73       180
weighted avg       0.74      0.74      0.74       180

[[44 25]
 [21 90]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.70      0.57      0.62        69
           1       0.76      0.85      0.80       111

    accuracy                           0.74       180
   macro avg       0.73      0.71      0.71       180
weighted avg       0.73      0.74      0.73       180

[[39 30]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [107]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Birthday_year','Title','Marital_Status']:
        data = data_.drop([col,'Is_young','Birthday_year','Title','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.80      0.75      0.78        69
           1       0.85      0.88      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[52 17]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.75      0.64      0.69        69
           1       0.79      0.86      0.83       111

    accuracy                           0.78       180
   macro avg       0.77      0.75      0.76       180
weighted avg       0.78      0.78      0.77       180

[[44 25]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowsk


[[55 15]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.57      0.67        70
           1       0.77      0.92      0.84       110

    accuracy                           0.78       180
   macro avg       0.79      0.74      0.76       180
weighted avg       0.79      0.78      0.77       180

[[ 40  30]
 [  9 101]]
               Time        Train          Test
model  0.087+/-0.01  0.993+/-0.0  0.818+/-0.02
Medical_Expenses_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.72      0.77        69
           1       0.84      0.90      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.81      0.82       180
weighted 

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.52      0.61        69
           1       0.75      0.88      0.81       111

    accuracy                           0.74       180
   macro avg       0.74      0.70      0.71       180
weighted avg       0.74      0.74      0.73       180

[[36 33]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.59      0.67      0.63        70
           1       0.77      0.71      0.74       110

    accuracy                           0.69       180
   macro avg       0.68      0.69      0.69       180
weighted avg       0.70      0.69      0.70       180

[[47 23]
 [32 78]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [108]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Title','Birthday_year','Parents or siblings infected']:
        data = data_.drop([col,'Is_young','Title','Birthday_year','Parents or siblings infected'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        69
           1       0.82      0.85      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        69
           1       0.82      0.85      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.78      0.78       180
weighted avg       0.79      0.79      0.79       180

[[49 20]
 [17 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '


[[54 16]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.63      0.72        70
           1       0.80      0.92      0.85       110

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 44  26]
 [  9 101]]
               Time        Train          Test
model  0.119+/-0.01  0.996+/-0.0  0.806+/-0.03
City
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.74      0.77        69
           1       0.85      0.89      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83     

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.73      0.59      0.66        69
           1       0.77      0.86      0.82       111

    accuracy                           0.76       180
   macro avg       0.75      0.73      0.74       180
weighted avg       0.76      0.76      0.76       180

[[41 28]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.68      0.66      0.67        70
           1       0.79      0.80      0.79       110

    accuracy                           0.74       180
   macro avg       0.73      0.73      0.73       180
weighted avg       0.74      0.74      0.74       180

[[46 24]
 [22 88]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              

In [110]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status','Is_young','Title','Parents or siblings infected']:
        data = data_.drop([col,'Birthday_year','Marital_Status','Is_young','Title','Parents or siblings infected'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.82      0.74      0.78        69
           1       0.85      0.90      0.87       111

    accuracy                           0.84       180
   macro avg       0.84      0.82      0.83       180
weighted avg       0.84      0.84      0.84       180

[[ 51  18]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.77      0.64      0.70        69
           1       0.80      0.88      0.84       111

    accuracy                           0.79       180
   macro avg       0.78      0.76      0.77       180
weighted avg       0.79      0.79      0.78       180

[[44 25]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cos

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.59      0.68        70
           1       0.78      0.92      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 41  29]
 [  9 101]]
               Time        Train          Test
model  0.082+/-0.02  0.993+/-0.0  0.803+/-0.02
City
---------

In [111]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Medical_Expenses_Family', 'City','Severity','Wife/Husband or children infected','Parents or siblings infected']:
        data = data_.drop([col,'Medical_Expenses_Family','City','Severity','Wife/Husband or children infected','Parents or siblings infected'],1)
        print(col)
        print(show_results(df,[model],k=4,scaler=RobustScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.79      0.71      0.75        69
           1       0.83      0.88      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.80      0.80       180
weighted avg       0.82      0.82      0.81       180

[[49 20]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.71      0.68      0.70        69
           1       0.81      0.83      0.82       111

    accuracy                           0.77       180
   macro avg       0.76      0.75      0.76       180
weighted avg       0.77      0.77      0.77       180

[[47 22]
 [19 92]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine'

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.79      0.78        70
           1       0.86      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [16 94]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.61      0.71        70
           1       0.79      0.93      0.85       110

    accuracy                           0.81       180
   macro avg       0.82      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 43  27]
 [  8 102]]
              Time        Train          Test
model  0.101+/-0.0  0.999+/-0.0  0.813+/-0.01
Gender
---------

In [112]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Infected_per_Family','Is_young','City','Marital_Status','Wife/Husband or children infected']:
        data = data_.drop([col,'Infected_per_Family','Is_young','City','Marital_Status','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.68      0.72        69
           1       0.82      0.87      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.78      0.78       180
weighted avg       0.80      0.80      0.80       180

[[47 22]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.67      0.71        69
           1       0.81      0.87      0.84       111

    accuracy                           0.79       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.79      0.79      0.79       180

[[46 23]
 [14 97]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '


[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        70
           1       0.86      0.86      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[55 15]
 [15 95]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.91      0.59      0.71        70
           1       0.79      0.96      0.87       110

    accuracy                           0.82       180
   macro avg       0.85      0.77      0.79       180
weighted avg       0.83      0.82      0.81       180

[[ 41  29]
 [  4 106]]
              Time        Train          Test
model  0.107+/-0.0  0.999+/-0.0  0.817+/-0

In [113]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Birthday_year','Title','Marital_Status','Parents or siblings infected']:
        data = data_.drop([col,'Is_young','Birthday_year','Title','Marital_Status','Parents or siblings infected'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.82      0.74      0.78        69
           1       0.85      0.90      0.87       111

    accuracy                           0.84       180
   macro avg       0.84      0.82      0.83       180
weighted avg       0.84      0.84      0.84       180

[[ 51  18]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.67      0.72        69
           1       0.81      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.78      0.79       180
weighted avg       0.80      0.81      0.80       180

[[46 23]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.59      0.68        70
           1       0.78      0.92      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 41  29]
 [  9 101]]
              Time        Train          Test
model  0.078+/-0.0  0.993+/-0.0  0.804+/-0.02
City
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.70      0.74        69
           1       0.82      0.88      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.81      0.81      0.81       

In [114]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Title','Birthday_year','Parents or siblings infected','Marital_Status']:
        data = data_.drop([col,'Is_young','Title','Birthday_year','Parents or siblings infected','Marital_Status'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.72      0.77        69
           1       0.84      0.90      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.81      0.82       180
weighted avg       0.83      0.83      0.83       180

[[ 50  19]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.68      0.73        69
           1       0.82      0.89      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.79      0.79       180
weighted avg       0.81      0.81      0.81       180

[[47 22]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric

[[48 21]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.76      0.79      0.77        70
           1       0.86      0.85      0.85       110

    accuracy                           0.82       180
   macro avg       0.81      0.82      0.81       180
weighted avg       0.82      0.82      0.82       180

[[55 15]
 [17 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.82      0.59      0.68        70
           1       0.78      0.92      0.84       110

    accuracy                           0.79       180
   macro avg       0.80      0.75      0.76       180
weighted avg       0.79      0.79      0.78       180

[[ 41  29]
 [  9 101]]
               Time        Train          Test
model  0.079+/-0.01  0.993+/-0.0  0.803+/-

In [115]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status','Is_young','Title','Parents or siblings infected','Infected_per_Family']:
        data = data_.drop([col,'Birthday_year','Marital_Status','Is_young','Title','Parents or siblings infected','Infected_per_Family'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.85      0.75      0.80        69
           1       0.86      0.92      0.89       111

    accuracy                           0.86       180
   macro avg       0.85      0.84      0.84       180
weighted avg       0.86      0.86      0.85       180

[[ 52  17]
 [  9 102]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.64      0.72        69
           1       0.80      0.91      0.85       111

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 44  25]
 [ 10 101]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.63      0.72        70
           1       0.80      0.92      0.85       110

    accuracy                           0.81       180
   macro avg       0.81      0.77      0.78       180
weighted avg       0.81      0.81      0.80       180

[[ 44  26]
 [  9 101]]
               Time        Train          Test
model  0.076+/-0.01  0.992+/-0.0  0.808+/-0.01
City
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.79      0.65      0.71        69
           1       0.80      0.89      0.85       111

    accuracy                           0.80       180
   macro avg       0.80      0.77      0.78       180
weighted avg       0.80      0.80      0.80     

In [116]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Infected_per_Family','Is_young','City','Marital_Status','Wife/Husband or children infected','Gender']:
        data = data_.drop([col,'Infected_per_Family','Is_young','City','Marital_Status','Wife/Husband or children infected','Gender'],1)
        print(col)
        print(show_results(df,[model],k=1,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.83      0.86      0.85       111

    accuracy                           0.81       180
   macro avg       0.80      0.79      0.79       180
weighted avg       0.80      0.81      0.80       180

[[49 20]
 [15 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        69
           1       0.81      0.88      0.84       111

    accuracy                           0.80       180
   macro avg       0.79      0.77      0.78       180
weighted avg       0.80      0.80      0.80       180

[[46 23]
 [13 98]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.77      0.81        70
           1       0.86      0.91      0.88       110

    accuracy                           0.86       180
   macro avg       0.85      0.84      0.85       180
weighted avg       0.85      0.86      0.85       180

[[ 54  16]
 [ 10 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.91      0.57      0.70        70
           1       0.78      0.96      0.86       110

    accuracy                           0.81       180
   macro avg       0.84      0.77      0.78       180
weighted avg       0.83      0.81      0.80       180

[[ 40  30]
 [  4 106]]
               Time        Train         Test
model  0.109+/-0.01  0.999+/-0.0  0.82+/-0.02
Medical_Expe

In [117]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Birthday_year','Title','Marital_Status','Parents or siblings infected','Wife/Husband or children infected']:
        data = data_.drop([col,'Is_young','Birthday_year','Title','Marital_Status','Parents or siblings infected','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.81      0.74      0.77        69
           1       0.85      0.89      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.82      0.82       180
weighted avg       0.83      0.83      0.83       180

[[51 18]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'mink

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.87      0.59      0.70        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.77      0.78       180
weighted avg       0.82      0.81      0.80       180

[[ 41  29]
 [  6 104]]
              Time        Train         Test
model  0.085+/-0.0  0.989+/-0.0  0.82+/-0.02
Infected_per_Family
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.68      0.75        69
           1       0.82      0.92      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.80      0.81       180
weighted avg       0.83      0.83    

In [118]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Title','Birthday_year','Parents or siblings infected','Marital_Status','Wife/Husband or children infected']:
        data = data_.drop([col,'Is_young','Title','Birthday_year','Parents or siblings infected','Marital_Status','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        69
           1       0.84      0.89      0.86       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       180

[[50 19]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.70      0.74        69
           1       0.82      0.89      0.86       111

    accuracy                           0.82       180
   macro avg       0.81      0.79      0.80       180
weighted avg       0.82      0.82      0.81       180

[[48 21]
 [12 99]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': '

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.77      0.81      0.79        70
           1       0.88      0.85      0.86       110

    accuracy                           0.83       180
   macro avg       0.82      0.83      0.83       180
weighted avg       0.84      0.83      0.83       180

[[57 13]
 [17 93]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.87      0.59      0.70        70
           1       0.78      0.95      0.86       110

    accuracy                           0.81       180
   macro avg       0.83      0.77      0.78       180
weighted avg       0.82      0.81      0.80       180

[[ 41  29]
 [  6 104]]
               Time        Train          Test
model  0.092+/-0.01  0.989+/-0.0  0.822+/-0.01
Infected_per_F

In [119]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent', 'Birthday_year','Marital_Status','Is_young','Title','Parents or siblings infected','Infected_per_Family','Wife/Husband or children infected']:
        data = data_.drop([col,'Birthday_year','Marital_Status','Is_young','Title','Parents or siblings infected','Infected_per_Family','Wife/Husband or children infected'],1)
        print(col)
        print(show_results(df,[model],method='stat', parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.75      0.79        69
           1       0.85      0.90      0.88       111

    accuracy                           0.84       180
   macro avg       0.84      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[ 52  17]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.70      0.76        69
           1       0.83      0.92      0.87       111

    accuracy                           0.83       180
   macro avg       0.84      0.81      0.82       180
weighted avg       0.83      0.83      0.83       180

[[ 48  21]
 [  9 102]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'me

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.85      0.57      0.68        70
           1       0.77      0.94      0.85       110

    accuracy                           0.79       180
   macro avg       0.81      0.75      0.77       180
weighted avg       0.80      0.79      0.78       180

[[ 40  30]
 [  7 103]]
              Time        Train          Test
model  0.07+/-0.01  0.988+/-0.0  0.817+/-0.01
Gender
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.66      0.67      0.66        69
           1       0.79      0.78      0.79       111

    accuracy                           0.74       180
   macro avg       0.72      0.73      0.72       180
weighted avg       0.74      0.74      0.74     

In [120]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Birthday_year','Title','Marital_Status','Parents or siblings infected','Wife/Husband or children infected','Infected_per_Family']:
        data = data_.drop([col,'Is_young','Birthday_year','Title','Marital_Status','Parents or siblings infected','Wife/Husband or children infected','Infected_per_Family'],1)
        print(col)
        print(show_results(df,[model],method='stat',remove=True,std=4,parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.82      0.71      0.76        69
           1       0.83      0.90      0.87       111

    accuracy                           0.83       180
   macro avg       0.82      0.81      0.81       180
weighted avg       0.83      0.83      0.83       180

[[ 49  20]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 90, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.84      0.70      0.76        69
           1       0.83      0.92      0.87       111

    accuracy                           0.83       180
   macro avg       0.84      0.81      0.82       180
weighted avg       0.83      0.83      0.83       180

[[ 48  21]
 [  9 102]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.79      0.79        70
           1       0.86      0.87      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[55 15]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.74      0.53      0.62        70
           1       0.75      0.88      0.81       110

    accuracy                           0.74       180
   macro avg       0.74      0.71      0.71       180
weighted avg       0.74      0.74      0.73       180

[[37 33]
 [13 97]]
              Time        Train          Test
model  0.062+/-0.0  0.989+/-0.0  0.811+/-0.04
Gender
-----------------

In [121]:
for col in data_.columns:
    if col not in ['Name','Medical_Tent','Is_young','Title','Birthday_year','Parents or siblings infected','Marital_Status','Wife/Husband or children infected','Infected_per_Family']:
        data = data_.drop([col,'Is_young','Title','Birthday_year','Parents or siblings infected','Marital_Status','Wife/Husband or children infected','Infected_per_Family'],1)
        print(col)
        print(show_results(df,[model],k=2,remove=True,std=4,scaler=MaxAbsScaler(),parameters=[parameter_space]))

Family_Case_ID
----------------------------------
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.83      0.75      0.79        69
           1       0.85      0.90      0.88       111

    accuracy                           0.84       180
   macro avg       0.84      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[ 52  17]
 [ 11 100]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.84      0.68      0.75        69
           1       0.82      0.92      0.87       111

    accuracy                           0.83       180
   macro avg       0.83      0.80      0.81       180
weighted avg       0.83      0.83      0.82       180

[[ 47  22]
 [  9 102]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric

{'weights': 'distance', 'p': 1, 'n_neighbors': 70, 'metric': 'minkowski', 'algorithm': 'kd_tree'}
              precision    recall  f1-score   support

           0       0.80      0.79      0.79        70
           1       0.86      0.87      0.87       110

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

[[55 15]
 [14 96]]
{'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'cosine', 'algorithm': 'brute'}
              precision    recall  f1-score   support

           0       0.82      0.53      0.64        70
           1       0.76      0.93      0.83       110

    accuracy                           0.77       180
   macro avg       0.79      0.73      0.74       180
weighted avg       0.78      0.77      0.76       180

[[ 37  33]
 [  8 102]]
               Time        Train          Test
model  0.068+/-0.01  0.989+/-0.0  0.817+/-0.03
Gender
-----------

## Best Model

In [124]:
parameter_space = {
    'n_neighbors':[10,20,30,40,50,60,70,80,90,100],
    'weights':['distance'],
    'algorithm':['ball_tree','kd_tree','brute'],
    'metric':['minkowski','cosine'],
    'p':[1]
}

model = KNeighborsClassifier()

In [141]:
X_train = data_.drop(['Name','Birthday_year','Parents or siblings infected','Infected_per_Family','Marital_Status','Title','Is_young'],1)
y_train = target

In [142]:
# deal with the null values
X_train = null_values(X_train,method='stat')

# deal with outliers
X_train, y_train = outliers(X_train,y_train,remove=True,std=4)

# scale the data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
        
# find the best hyperparameters
clf = GridSearchCV(model,parameter_space,n_jobs=-1,scoring='accuracy')
clf.fit(X_train,y_train)
model = clf.best_estimator_
print(clf.best_params_)

# fit the model to the data
model.fit(X_train,y_train)

# predictions
Y_pred = model.predict(X_train)

# check the mean accuracy for the train
value_train = model.score(X_train,y_train)

# print a classification report and confusion matrix
print(classification_report(y_true=y_train,y_pred=Y_pred))
print(confusion_matrix(y_true=y_train,y_pred=Y_pred))

{'algorithm': 'ball_tree', 'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'distance'}
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       339
           1       1.00      0.99      0.99       538

    accuracy                           0.99       877
   macro avg       0.99      0.99      0.99       877
weighted avg       0.99      0.99      0.99       877

[[338   1]
 [  6 532]]


## Predicting new values

In [145]:
test = pd.read_csv('test.csv')

In [146]:
test['Infected_per_Family'] = test['Parents or siblings infected'] + test['Wife/Husband or children infected']
test['Marital_Status'] = 0
test['Gender'] = 0
test['Title'] = 0
# 2020-60 = 1960
test['Is_young'] = 0
for i in test.index:
    if test['Birthday_year'][i]<=1960:
        test['Is_young'][i] = 0
    elif test['Birthday_year'][i]>1960:
        test['Is_young'][i] = 1
    else:
        test['Is_young'][i] = None

for i in test.index:
    if test['Name'][i].startswith('Miss'):
        test['Marital_Status'][i] = 0
        test['Gender'][i] = 1
        test['Title'][i] = 'Miss'
        
    elif test['Name'][i].startswith('Ms.'):
        test['Marital_Status'][i] = None
        test['Gender'][i] = 1
        test['Title'][i] = 'Ms.'
        
    elif test['Name'][i].startswith('Mrs.'):
        test['Marital_Status'][i] = 1
        test['Gender'][i] = 1
        test['Title'][i] = 'Mrs.'
        
    elif test['Name'][i].startswith('Mr.'):
        test['Marital_Status'][i] = 1
        test['Gender'][i] = 0
        test['Title'][i] = 'Mr.'
        
    elif test['Name'][i].startswith('Master'):
        test['Marital_Status'][i] = 0
        test['Gender'][i] = 0
        test['Title'][i] = 'Master'

In [147]:
test['City'] = le_city.transform(test['City'])
test['Title'].loc[test['Title'] != 'Mrs.'] = le_title.transform(test['Title'].loc[test['Title'] != 'Mrs.'])

In [148]:
test['Title'].loc[test['Title'] == 'Mrs.'] = 4

In [157]:
X_test = test.drop(['Patient_ID','Name','Birthday_year','Parents or siblings infected','Infected_per_Family','Marital_Status','Title','Is_young'],1)

In [158]:
X_test.head()

,Family_Case_ID,Severity,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Gender
0,49242,3,0,203,NaN,1,0
1,10400,3,0,631,NaN,1,0
2,10795,3,0,376,NaN,0,0
3,62440,3,1,405,NaN,0,0
4,81311,2,0,378,NaN,1,0


In [159]:
# deal with the null values
X_test = null_values(X_test,k=2)

In [160]:
pred = model.predict(X_test)

In [161]:
prediction = pd.DataFrame({'Patient_ID':test['Patient_ID'],'Deceased':pred})

In [162]:
prediction

,Patient_ID,Deceased
0,901,1
1,902,1
2,903,1
3,904,1
4,905,1
...,...,...
395,1296,1
396,1297,1
397,1298,0
398,1299,1


In [163]:
prediction.to_csv('instancebased.csv',index=False)